### import

In [4]:
import os
import shutil
import time
import pandas as pd
import numpy as np
from tqdm import tqdm      #tqdm(filelist, total = len(file_list), position=0, leave=True)
import re
import traceback
from os.path import join
import file_function as ff
from file_function import read_filelist, re_name_upload, write_log_csv_upload
from datetime import datetime
import pickle

path_success = r'D:\1.전산,개발\3.파일서버관리\구스캔폴더 작업log\success'
path_noDB = r"D:\1.전산,개발\3.파일서버관리\구스캔폴더 작업log\noDB"
path_pkl_dir = r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회"

# 데이터 로드######################################
## 모두 object다 str. 사용가능, 주민번호는 _ 포함되어있음
debt_pkl = "채무자조회새창_20221220_1128"
grt_pkl = "보증인새창_20221222_1650(파일명작업을 위해 새로 받음)"
lawact_pkl = "법조치조회새창_20230621_0914스캔파일작업용"
rehabilitation_pkl = "개인회생새창_20230313_1526"
acc_pkl = "계좌조회새창_20230223_0839"
##################################################
df_debt_ori = pd.read_pickle(join(path_pkl_dir, debt_pkl+".pkl")) #성명, 주민번호인, 매각사구분, 매각사, 원채권사 (아래서도 공통) # 현재원금 순으로 정렬됨
df_grt_ori = pd.read_pickle(join(path_pkl_dir, grt_pkl+".pkl")) #보증인성명인, 주민번호인, [채무자명, 채무자주민번호]
df_law_all_ori = pd.read_pickle(join(path_pkl_dir, lawact_pkl+".pkl"))#[["채무자키", "대상자명", "매각사구분", "매각사","원채권사", "법조치키","관할법원", "사건번호", "관련사건번호", "관련법조치사건번호"]] #관할법원, 사건번호, 관련사건번호, #대상자명, 보증인명 (원채권사x)... 

# int > str "str" 따옴표 쓰면 안 됨!!!!!!!!!!!!!! ################################################
df_law_all_ori["채무자키"] = df_law_all_ori.채무자키.apply(str)

df_rehabilitation_ori = pd.read_pickle(join(path_pkl_dir, rehabilitation_pkl+".pkl"))[["채무자키","분납자성명인", "매각사구분", "OPB", "관할법원", "사건번호"]]
df_acc_ori = pd.read_pickle(join(path_pkl_dir, acc_pkl+".pkl")) 
df_duplbirth = pd.read_pickle(r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회\기타\생일동일.pkl")

def getMtime(path):
    """
    (pdf)수정일=최초생성일!! 이거 써(이동/파일명 변경해도 안 바뀜)
    path : (walk)dir + file 또는 (listdir) file
    """
    a = time.ctime(os.path.getmtime(path))
    b = time.strptime(a)
    c = time.strftime('%y%m%d', b)
    return c

In [5]:
df_debt = df_debt_ori.copy()
df_grt = df_grt_ori.copy()
df_law = df_law_all_ori.copy()
df_rehabilitation = df_rehabilitation_ori.copy()
df_acc = df_acc_ori.copy()

### 되돌리기(로그파일)

In [17]:
###############
path_log = r"D:\1.전산,개발\3.파일서버관리\구스캔폴더 작업log\success\20240326 170342_45_광주 양도통지서스캔 양통종적 양통종적스캔 한울.csv"
###############

rollback = pd.read_csv(path_log, encoding='utf', sep=',', names=["old_dir", "old_file", "new_dir", "new_file"])

for i, v in rollback.iterrows() :
    os.rename(join(v.new_dir, v.new_file), join(v.old_dir, v.old_file))


### 수정일 붙이기

In [3]:
path = r"D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\신창(수작업)\신창지사\기타\황명숙\공유폴더2\양도통지서 스캔 (파일 좀 있음)\디엔피 양도통지서 스캔\디엔피 양통2차 보증인\새 폴더"
for f in read_filelist(path) :
    new_f = os.path.splitext(f)[0] + " " + getMtime(join(path, f)) + os.path.splitext(f)[1]
    os.rename(join(path, f), join(path, new_f))

### 엑셀(list파일) 읽어서 이름바꾸기

In [15]:
# 엑셀에 콜론(:) 지우기 (파일명에 쓸 수 없는 기호) ex - 변경전:김향자
path_dir = r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회\extraKey"
path_file = "우담.xlsx"
df_filename = pd.read_excel(join(path_dir, path_file)).fillna("") #dropna(axis=0, how="any")
print(len(df_filename))
display(df_filename.head(1))
#display(df_filename.info())
# 번호가 1.0으로 나와도, 파일명의 번호를 int로 바꾸면 비교에 문제가 없다.

6689


,Unnamed: 0,일련번호,채무자키
0,0,10801648,20459822


In [17]:
########################################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\11.환매\환매통지서\스캔_차주"
addInfo = "서울 채권별 20.우담"
add_file_name = ""
path_noDB = join("D:\전산부재", addInfo) ### 미양도 유의
########################################################
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
success, fail, noDB = [], [], []
file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    pk = re.search("\d{8,8}", stem).group()
                                 ######################
    temp = df_filename[df_filename["일련번호"]==int(pk)]  # int() 주의
    if len(temp) == 1 :
        try :
            # # # 양통과 종적이 나란히 엑셀파일에 기록된 경우(세종-한울) ######################
            # # # if temp.수신인.values[0] != "" : # 이름 있는 경우, 양통
            # # #     new_stem = re.sub("\*", "", temp["주민번호"].values[0]) + " " + re.sub("\([주유]\)", "", temp["수신인"].values[0]) + " 양통 세종 석영"
            # # # else : # 종적조회
            # # #     temp = df_filename[df_filename['번호'] == int(pk)-1]
            # # #     if len(temp) ==1 :
            # # #         new_stem = re.sub("\*", "", temp["주민번호"].values[0]) + " " + re.sub("\([주유]\)", "", temp["수신인"].values[0]) + " 양통 종적조회 세종 석영"
            # # #     else : # 양통이 없는경우
            # # #         fail.append([path, f, "no양통"])
            # # #         continue  ######################


                                        ############                                                ###########             
            # new_stem = re.sub("\*", "", temp["주민번호"].values[0]) + " " + re.sub("\([주유]\)", "", temp["수신인"].values[0]) + add_file_name
            # new_name = stem + " " + new_stem + ext
            
            new_stem = str(temp["채무자키"].values[0]) + " " + stem
            new_name = new_stem + ext

            os.rename(join(path, f), join(path, new_name))
            success.append([path, f, new_name])
        except :
            fail.append([path, f, new_name, "파일명 변경중 에러"])
            continue
    elif len(temp) ==0  :
        os.rename(join(path, f), join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
    else :
        fail.append([path, f, "multi"])

print(f"{total - len(success) - len(fail) - len(noDB)}개 파일 누수")
print(f"{total}개 중 {len(success)}개 파일명변경, {len(fail)}개 변경실패, {len(noDB)}개 전산없음")
write_log_csv_upload(success, path_success, addInfo+"성명")
write_log_csv_upload(fail, path, "fail성명")
write_log_csv_upload(noDB, path_noDB, "전산없음(양도채권x)")

100%|██████████| 179/179 [00:00<00:00, 2088.35it/s]

0개 파일 누수
179개 중 173개 파일명변경, 3개 변경실패, 3개 전산없음


### 사건번호로 채무자키 찾기
- 이름과 사건번호로 검색(관할법원이 다르고 사건번호 같은 경우 대비) > 전산 없는 경우 사건번호로만 > 수작업
- 매각사 제한은 바로 아래 칸에서

In [14]:
df_law_all.columns

Index(['채무자키', '대상자명', '매각사구분', '매각사', '원채권사', '법조치키', '관할법원', '사건번호',
       '관련사건번호', '관련법조치사건번호'],
      dtype='object')

In [9]:
# 매각사 등 조건 걸기
df_law = df_law_all#[df_law_all.매각사구분 == '한울가람']

# # 개인회생
# df_rehabilitation["관련사건번호"] = ""
# df_rehabilitation["관련법조치사건번호"] = ""
# df_rehabilitation.rename(columns={"분납자성명인" : "대상자명"}, inplace=True)
# df_law = df_rehabilitation

print(len(df_law.index))

154959


In [ ]:
# # 사건번호 있는 파일만 따로 모으기
# path = r"D:\구 스캔파일\광주 공유폴더\통합파일\파산 개인회생 신용회복\개회.파산.(신복)"
# p_e = re.compile(r"(\d{2,4})\s?([가간나느다머차카타즈본징하개회라정][가-힣]{0,2})\s?(\d+)")
# for f in read_filelist(path) :
#     if p_e.search(f) : os.rename(join(path, f), join(path, "사건번호", f))

In [10]:
#################################
path = r"D:\구 스캔파일\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울가람 판결문,결정문"
nameSearching = False # T/F  #############################
addInfo = "광주 통합 2.한울"
add_file_name = ""

p_e = re.compile(r"(\d{2,4})\s?([가간나느다머차카타즈본징하개회라정][가-힣]{0,2})\s?(\d+)")
p_name = re.compile("[가-힣]{2,}")

path_noexp = join(path, "noexp")
path_multi = join(path, "multi")
path_noDB = join(r'D:\전산부재', addInfo)
if nameSearching : path_completed = join(r"D:\업로드완료", addInfo)
else : path_completed = path # 검토해야 하는 것은 제자리에
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
if not os.path.exists(path_noexp) : os.mkdir(path_noexp)
if not os.path.exists(path_multi) : os.mkdir(path_multi)
#################################

file_list = read_filelist(path)
total = len(file_list)
success, noDB, fail, multi = [], [], [], []
cnt_noexp = 0

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)
    
    # 파일명에 정규식 있는지
    exp_result = p_e.search(f)
    if exp_result == None : 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    else : 
        # 연도수 맞추기
        if len(exp_result[1]) == 2 :
            if int(exp_result[1])>50 :
                event = "19" + exp_result[1]+exp_result[2]+exp_result[3]
            else : 
                event = "20" + exp_result[1]+exp_result[2]+exp_result[3]    
        else : event = exp_result[1]+exp_result[2]+exp_result[3]
    
    # 대상자명 검색 여부
    if nameSearching :
        if p_name.search(f) :
            name = p_name.search(f).group()
        else : name = "" # 언제나 True가 된다.
        # 전산이 잘못된 경우 있어서 연도는 4자리 2자리 모두 확인
        # 띄어쓰기는 전산에서도 없앴으니 문제될 거 없다.
        # df에서 채무자키 찾기 : 연도를 느슨하게 했기 때문에 contains로 해봐야 일련번호 다른 사건이 검색되므로 일치값만 찾는다.
        # 대상자명을 함께 찾으면 중복인 경우는 대부분 해결, 단 관련사건번호에만 있는 경우는 검색조건에서 탈락하게 되는 문제가 있으니 수가 많은 경우 이름 제외하고 재검색.(다른 사건번호가 검색될 일은 없겠다.)
        keys_df = df_law[((df_law.사건번호==event)|(df_law.사건번호==event[2:])|(df_law.관련사건번호==event)|(df_law.관련사건번호==event[2:])|(df_law.관련법조치사건번호==event)|(df_law.관련법조치사건번호==event[2:])) & (df_law.대상자명.str.contains(name))].drop_duplicates(subset=["채무자키"], keep='first')
        keys = keys_df.채무자키.values
    else :
        keys_df = df_law[((df_law.사건번호==event)|(df_law.사건번호==event[2:])|(df_law.관련사건번호==event)|(df_law.관련사건번호==event[2:]))].drop_duplicates(subset=["채무자키"], keep='first')
        keys = keys_df.채무자키.values
    
    if len(keys) ==1 : # 전산에 사건번호가 잘못 올라가는 경우도 있어서 연도는 2자리, 4자리로 모두 검색해야 할듯
        new_name = str(keys[0]) + " " + f
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    elif len(keys) == 0 : 
        #os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f]) #이제는 noDB로(다시 찾아보긴 할거다.)
    else : 
        suffix = ""
        for i in range(len(keys_df)) :
            suffix += " ".join(keys_df[["채무자키","관할법원"]].values[i]) + " "
        new_name = suffix + f
        os.rename(src, join(path_multi, new_name))
        multi.append([path, f, path_multi, new_name])#### key넣어뒀으니 이거 보면서 찾자

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(fail, path, "fail")
write_log_csv_upload(multi, path_multi, "multi")
write_log_csv_upload(noDB, path_noDB, "noDB")

loss = total - len(success) - len(noDB) - len(fail) - len(multi) - cnt_noexp
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 결과 여러건 : {len(multi)}개, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}")

100%|██████████| 18/18 [00:00<00:00, 64.98it/s]

log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울가람 판결문,결정문
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울가람 판결문,결정문\multi
파일누수 0개
총 파일 : 18, 파일명 변경 : 7개, 결과 여러건 : 0개, 파일명 변경 실패 : 0개, 전산없음 : 11개, 정규식없음 : 0


### 이름-주민번호 등

##### [핵심] 이름, 주민번호, 차주or보증인, 원채권사 > 양통류
- 주민번호는 -이 있는 경우 -을 포함할 것(123456-1, 123-12-1, 123456)

In [14]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\종결.매각.환매\종결, 매각(양통)"
addInfo = "광주 종결 종결 (양통)"
add_file_name = " 양통"

debt_or_grt = 2 # 차주만 : 0, 보증인만 : 1, 둘다검색 : 2 ###################

# 리팩토링 한다면 매칭과 contains를 구분해서 딕셔너리 만들고, 그 개수만큼 쿼리문 += 반복해서 만들면 될듯
col_1 = "주민번호인" # match # 사업자번호중 마지막 하나가 안 붙어있는 경우도 있네.{8,8}>{7,8}
p_1 = re.compile("[\d-]{6,8}") 
col_2 = "성명" # contains "보증인성명인"
p_2 = re.compile("[가-힣]{2,}")
col_3 = "매각사구분" # "매각사구분" # match # sw : serching-word
sw_3 = "DNP" # "" 
col_4 = "원채권사" # contains 
sw_4 = "" #p_4 = re.compile(r"무룡") #search니까 제한두지 않을거면 공백으로
###########################################

path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
path_noexp = join(path, "noexp")
path_multi = join(path, 'multi')
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
if not os.path.exists(path_noexp) : os.mkdir(path_noexp)
if not os.path.exists(path_multi) : os.mkdir(path_multi)

#log
# 검색 결과 여러건인경우
success, noDB, fail, multi = [], [], [], []
cnt_noexp = 0

file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)

    if p_1.search(f) and p_2.search(f) :
        sw_1 = p_1.search(f).group()
        sw_2 = p_2.search(f).group()
    else : 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    
    # 1) df 검색
    if debt_or_grt == 0 :
        debt_result = df_debt[(df_debt[col_1].str.match(sw_1)) & (df_debt[col_2].str.contains(sw_2)) & (df_debt[col_3].str.match(sw_3)) & (df_debt[col_4].str.contains(sw_4))]
        set_keys = debt_result.drop_duplicates(subset=["채무자키"], keep='first').채무자키.values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로
    elif debt_or_grt == 1 :
        grt_result = df_grt[(df_grt[col_1].str.match(sw_1)) & (df_grt["보증인성명인"].str.contains(sw_2)) & (df_grt[col_3].str.match(sw_3)) & (df_grt[col_4].str.contains(sw_4))]
        set_keys = grt_result.drop_duplicates(subset=["채무자키"], keep='first').채무자키.values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로
    elif debt_or_grt == 2 : 
        debt_result = df_debt[(df_debt[col_1].str.match(sw_1)) & (df_debt[col_2].str.contains(sw_2)) & (df_debt[col_3].str.match(sw_3)) & (df_debt[col_4].str.contains(sw_4))]
        grt_result = df_grt[(df_grt[col_1].str.match(sw_1)) & (df_grt["보증인성명인"].str.contains(sw_2)) & (df_grt[col_3].str.match(sw_3)) & (df_grt[col_4].str.contains(sw_4))]

        debt_keys = debt_result.drop_duplicates(subset=["채무자키"], keep='first').채무자키.values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로
        grt_keys = grt_result.drop_duplicates(subset=["채무자키"], keep='first').채무자키.values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로
        set_keys = set(debt_keys).union(set(grt_keys))
    else : 
        print("차주인지 보증인인지 선택하세요")
        break 
        
    # 2) 검색결과 개수에 따라 채무자키 확정 및 이름 변경
    if len(set_keys) == 0 :
        os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
    elif len(set_keys) ==1 :
        set_keys = tuple(set_keys) # 인덱싱 위해 튜플로
        new_name = str(set_keys[0]) + " " + stem + add_file_name + ext
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    else : # 여러 건
        multi.append([path, f, set_keys])
        os.rename(src, join(path_multi, f))

loss = total - len(success) - len(noDB) - len(fail) - cnt_noexp -len(multi)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, multi수작업 : {len(multi)}, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}")

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(noDB, path_noDB, addInfo)
write_log_csv_upload(fail, path, "fail")


100%|██████████| 17/17 [00:01<00:00, 15.66it/s]

파일누수 0개
총 파일 : 17, 파일명 변경 : 16개, multi수작업 : 0, 파일명 변경 실패 : 0개, 전산없음 : 1개, 정규식없음 : 0
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\종결.매각.환매\종결, 매각(양통)


In [13]:
set_keys

{'20414654'}

##### 이름,생일(초본,회생,파산) > 복사
- 주민번호 앞자리만 있어도 전체 있는 것과 결과는 같다.(예외 한 건)
- 이름, 생일 같고 다른 사람인 경우 df_duplbirth에 정리해놨다. 이 경우 아니고, 매각사나 원채권자 다르면 복사

In [8]:
# 범위 제한 : 복사하니까 등초본은 할 필요가 없다.
df_debt = df_debt[df_debt.매각사구분 == "케이에스"]
df_grt = df_grt[df_grt.매각사구분 == "케이에스"]

In [3]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\신창양통종적조회(대부분 종결 나중에 수작업)"
addInfo = "광주 양도통지서스캔 양통종적 신창양통"
add_file_name = " 양통 종적조회"

# 사업자12(10), 자연인14(13)자리 하이픈 있냐없냐에 따라 코드 달리해야함
p_birth = re.compile("[0-9]{6}")
p_name = re.compile("[가-힣]{2,}")

path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
path_noexp = join(path, "noexp")
path_multi = join(path, 'multi')
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
if not os.path.exists(path_noexp) : os.mkdir(path_noexp)
if not os.path.exists(path_multi) : os.mkdir(path_multi)
###########################################

success, noDB, fail, multi = [], [], [], []
cnt_noexp, conpy_origin_cnt, copy_cnt = 0, 0, 0

file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)

    # 파일명에 정규식 있는지
    exp_result1 = p_name.search(f)
    exp_result2 = p_birth.search(f)

    if (exp_result1 == None) or (exp_result2 == None) : 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    else : 
        # 연도수 맞추기
        name = exp_result1[0]
        birth = exp_result2[0]

    ###### 매각사와 원채권사가 같으면 동일채무로 보고 현재원금 큰거 하나만 남긴다.(정렬되어있음)
    keys_debt = df_debt[(df_debt.성명.str.contains(name)) & (df_debt.주민번호인.str.match(birth))].drop_duplicates(subset=["채무자키"], keep='first')
    keys_debt = keys_debt.drop_duplicates(subset=["매각사", "원채권사"], keep='first').채무자키.values
    keys_grt = df_grt[(df_grt.보증인성명인.str.contains(name)) & df_grt.주민번호인.str.match(birth)].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    keys = np.append(keys_debt, keys_grt)
    keys_set = set(keys)

    if len(keys) == 0 : 
        os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
    elif len(keys) ==1 :
        new_name = str(keys[0]) + " " + stem + add_file_name + ext
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    else : # 여러 건 - 그냥 수동으로 하자... 채무자와 보증인 각각 있는 경우 복사
        if len(df_duplbirth[(df_duplbirth["성명"]==name) & (df_duplbirth["생년"] == birth[:6])]) > 0 : # 다른사람.. 수작업해야함
            os.rename(src, join(path_multi, f))
            multi.append([path, f, keys_set])
        else :
            for key in keys :
                new_name = str(key) + " " + stem + add_file_name + ext
                if not os.path.exists(join(path_completed, "복사건")) : os.mkdir(join(path_completed, "복사건")) ###############
                shutil.copy(join(path,f), join(path_completed, "복사건", new_name))
                success.append([path, f, path_completed, new_name, "복사건"])
                copy_cnt += 1
            conpy_origin_cnt += 1
            os.remove(join(path, f))

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(noDB, path_noDB, "noDB")
write_log_csv_upload(fail, path, "fail")

loss = total - (len(success) - (copy_cnt - conpy_origin_cnt)) - len(noDB) - len(fail) - cnt_noexp -len(multi)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success) - (copy_cnt - conpy_origin_cnt)}개, multi수작업 : {len(multi)}, \
파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}, 복사건 : {conpy_origin_cnt, copy_cnt}")

  0%|          | 0/447 [00:00<?, ?it/s]

100%|██████████| 447/447 [00:00<00:00, 798.44it/s]

log 파일이 비어있습니다. path : D:\전산부재\광주 양도통지서스캔 양통종적 신창양통
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\신창양통종적조회(대부분 종결 나중에 수작업)
파일누수 0개
총 파일 : 447, 파일명 변경 : 6개, multi수작업 : 0, 파일명 변경 실패 : 0개, 전산없음 : 0개, 정규식없음 : 441, 복사건 : (0, 0)


In [4]:
file_list

['구혜경(730213)  .pdf',
 '권미란-.pdf',
 '권용범(57)  .pdf',
 '권형태.pdf',
 '권홍관.pdf',
 '길영환 .pdf',
 '김경석.pdf',
 '김경자.pdf',
 '김경진-.pdf',
 '김광수.pdf',
 '김광현.pdf',
 '김근영.pdf',
 '김기성 .pdf',
 '김기성.pdf',
 '김기자.pdf',
 '김길수 .pdf',
 '김나윤 .pdf',
 '김덕권 (2).pdf',
 '김덕권 (3).pdf',
 '김덕권 (4).pdf',
 '김덕권.pdf',
 '김동현 .pdf',
 '김두영.pdf',
 '김령희 .pdf',
 '김명기.pdf',
 '김명숙.pdf',
 '김명순.pdf',
 '김명희.pdf',
 '김민석.pdf',
 '김민선 .pdf',
 '김민선.pdf',
 '김병윤.pdf',
 '김복염 .pdf',
 '김붕배.pdf',
 '김상봉.pdf',
 '김상진-.pdf',
 '김석환.pdf',
 '김성규 .pdf',
 '김성애.pdf',
 '김성원 .pdf',
 '김성주.pdf',
 '김성희.pdf',
 '김세철 .pdf',
 '김수근 .pdf',
 '김수용 .pdf',
 '김수현.pdf',
 '김숙희-.pdf',
 '김순옥 (2).pdf',
 '김순옥 (3).pdf',
 '김순옥.pdf',
 '김순용 .pdf',
 '김순희 .pdf',
 '김애순 .pdf',
 '김양겸.pdf',
 '김양호-.pdf',
 '김연석.pdf',
 '김연심.pdf',
 '김연옥.pdf',
 '김영.pdf',
 '김영권.pdf',
 '김영길.pdf',
 '김영남 (2).pdf',
 '김영남.pdf',
 '김영무 (2).pdf',
 '김영무.pdf',
 '김영숙.pdf',
 '김영식.pdf',
 '김영주.pdf',
 '김영화-.pdf',
 '김영희.pdf',
 '김옥남-.pdf',
 '김윤일-.pdf',
 '김은수.pdf',
 '김은현.pdf',
 '김인수.pdf',
 '김인숙 .pdf',
 '김일기 .pdf',
 '김재득(6

##### 이름,생일(초본 외) > multi : 수작업

In [33]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\신창양통종적조회(대부분 종결 나중에 수작업)"
addInfo = "광주 양도통지서스캔 양통종적 신창양통"
add_file_name = " 양통 종적조회"

# 사업자12(10), 자연인14(13)자리 하이픈 있냐없냐에 따라 코드 달리해야함
p_birth = re.compile("([0-9]{6})([0-9]{0,7})")
p_name = re.compile("[가-힣]{2,}")

path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
path_noexp = join(path, "noexp")
path_multi = join(path, 'multi')
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
if not os.path.exists(path_noexp) : os.mkdir(path_noexp)
if not os.path.exists(path_multi) : os.mkdir(path_multi)
###########################################

success, noDB, fail, multi = [], [], [], []
cnt_noexp = 0

file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)

    # 파일명에 정규식 있는지
    exp_result1 = p_name.search(f)
    exp_result2 = p_birth.search(f)

    if (exp_result1 == None) or (exp_result2 == None) : 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    else : 
        # 연도수 맞추기
        name = exp_result1[0]
        birth = exp_result2[1] + "-"+exp_result2[2]

    ###### 매각사와 원채권사가 같으면 동일채무로 보고 현재원금 큰거 하나만 남긴다.(정렬되어있음)
    keys_debt = df_debt[(df_debt.성명.str.contains(name)) & (df_debt.주민번호인.str.match(birth))].drop_duplicates(subset=["채무자키"], keep='first')
    keys_debt = keys_debt.drop_duplicates(subset=["매각사", "원채권사"], keep='first').채무자키.values
    keys_grt = df_grt[(df_grt.보증인성명인.str.contains(name)) & df_grt.주민번호인.str.match(birth)].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    keys = np.append(keys_debt, keys_grt)
    keys_set = set(keys)

    if len(keys) == 0 : 
        os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
    elif len(keys) ==1 :
        new_name = str(keys[0]) + " " + stem + add_file_name + ext
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    else : # 여러 건 - 채무자키는 하나고, 보증인에도 있을 때만 복사
        os.rename(src, join(path_multi, f))
        multi.append([path, f, keys_set])

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(noDB, path_noDB, "noDB")
write_log_csv_upload(fail, path, "fail")
write_log_csv_upload(multi, path_multi, "multi")

loss = total - len(success) - len(noDB) - len(fail) - len(multi) -cnt_noexp
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 결과 여러건 : {len(multi)}개, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}")

100%|██████████| 469/469 [00:01<00:00, 390.98it/s]

log 파일이 비어있습니다. path : D:\전산부재\광주 양도통지서스캔 양통종적 신창양통
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\신창양통종적조회(대부분 종결 나중에 수작업)
파일누수 0개
총 파일 : 469, 파일명 변경 : 7개, 결과 여러건 : 1개, 파일명 변경 실패 : 0개, 전산없음 : 0개, 정규식없음 : 461


### 단일 컬럼 검색(관리자기타...)

- 매각사 등의 조건 추가시 df 자체를 제한하면 됨

In [5]:
df_ref = df_debt[df_debt.매각사구분 == "대산"]
print(df_ref.columns)
print(len(df_ref.index))

Index(['채무자키', '매각사구분', '채권구분', '성명', '주민번호인', '담당자', '채무상태', '성별', '연령',
       '원채권사', '매각사', '최초원금', '최초합계', '현재원금', '관리자기타', '비고', '종결일', '보증인성명',
       '보증인상태', '타채무자키'],
      dtype='object')
618


In [7]:
#################################
path = r"D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인"
addInfo = "서울 채권별 7.대산대부 3.양통 1차 보증인"
add_file_name = " 양통" # 있으면 공백

col = "관리자기타" 
# p = re.compile("[가-힣]+") # 성명, 보증인성명
p = re.compile("\d{10}") # 관리자 기타
# p = re.compile("\d{7,8}\s\d{6,7}") # 녹음파일 년월일시분 = 계좌번호
####
path_completed = path # join(r"D:\업로드완료", addInfo)
path_noDB = join(r'D:\전산부재', addInfo) 
path_noexp = join(path, "noexp")
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
#################################

file_list = read_filelist(path)
total = len(file_list)
success, noDB, fail, multi = [], [], [], []
cnt_noexp = 0

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)
    
    # 파일명에 정규식 있는지
    exp_result = p.search(f)
    if exp_result == None : 
        if not os.path.exists(path_noexp) : os.mkdir(path_noexp) 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    else : 
        #########################################
        sn = exp_result[0] # 기본값 
        # sn = re.sub(" ", "", exp_result[0]) 
        ##########################################
    
    ##############################################################################################################################
    # 일치
    keys = df_ref[df_ref[col] == sn].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    # # 포함
    # keys = df_ref[(df_ref[col].str.contains(sn))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    ##############################################################################################################################

    if len(keys) ==1 : 
                                                            ################################ 수정일 
        new_name = str(keys[0]) + " " + stem + add_file_name # +" " + getMtime(join(path, f))
        new_name = new_name + ext
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    elif len(keys) == 0 : 
        os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f]) 
    else : 
        multi.append([f, keys])

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(fail, path, "fail")
write_log_csv_upload(multi, path, "multi")
write_log_csv_upload(noDB, path_noDB, "noDB")

loss = total - len(success) - len(noDB) - len(fail) - len(multi) - cnt_noexp
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 결과 여러건 : {len(multi)}개, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}")

100%|██████████| 508/508 [00:00<00:00, 1392.35it/s]

log 파일이 비어있습니다. path : D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인
log 파일이 비어있습니다. path : D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인
log 파일이 비어있습니다. path : D:\전산부재\서울 채권별 7.대산대부 3.양통 1차 보증인
파일누수 0개
총 파일 : 508, 파일명 변경 : 508개, 결과 여러건 : 0개, 파일명 변경 실패 : 0개, 전산없음 : 0개, 정규식없음 : 0


In [9]:
#################################
path = r"D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인\새 폴더"
addInfo = "서울 채권별 7.대산대부 3.양통 1차 보증인 (보증인이름넣기)"
#################################
success, fail = [], []
file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)
    key = stem[:8]
    
    grt_names = df_ref[df_ref["채무자키"] == key].보증인성명.values[0]
    new_name = stem + grt_names + ext
    
    try : 
        os.rename(src, join(path, new_name))
        success.append([path, f, path, new_name])
    except Exception as e :
        fail.append([f, new_name, e.__class__, e])
        continue

write_log_csv_upload(success, path, "success")
write_log_csv_upload(fail, path, "fail")

loss = total - len(success) - len(fail)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 파일명 변경 실패 : {len(fail)}개")

100%|██████████| 412/412 [00:00<00:00, 1984.32it/s]

log 파일이 비어있습니다. path : D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인\새 폴더
파일누수 0개
총 파일 : 412, 파일명 변경 : 412개, 파일명 변경 실패 : 0개


##### 이름만 있는 파일(차주/보증인)

In [18]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\양통종적 스캔(수작업)\한울가람 양통종적(수작업)\1차검색실패(수작업)"
addInfo = "광주 양도통지서스캔 양통종적 양통종적스캔 한울"
add_file_name = " 양통 종적조회"
## 제한할 수 있으면 #############################
df_debt = df_debt_ori.query('매각사구분.str.contains("한울|DNP")') # |한울가람
df_grt = df_grt_ori.query('매각사구분.str.contains("한울|DNP")')
###########################################

p_name = re.compile("[가-힣]{2,}")

path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
path_noexp = join(path, "noexp")
path_multi = join(path, 'multi')
if not os.path.exists(path_noDB) : os.makedirs(path_noDB)
if not os.path.exists(path_completed) : os.makedirs(path_completed)
if not os.path.exists(path_noexp) : os.mkdir(path_noexp)
if not os.path.exists(path_multi) : os.mkdir(path_multi)



success, noDB, fail, multi = [], [], [], []
cnt_noexp, conpy_origin_cnt, copy_cnt = 0, 0, 0

file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    stem, ext = os.path.splitext(f)

    # 파일명에 정규식 있는지
    exp_result1 = p_name.search(f)

    if exp_result1 == None : 
        os.rename(src, join(path_noexp, f))
        cnt_noexp += 1
        continue
    else :
        name = exp_result1[0]

    ###### 매각사와 원채권사가 같으면 동일채무로 보고 현재원금 큰거 하나만 남긴다.(정렬되어있음)
    keys_debt = df_debt[(df_debt.성명.str.contains(name))].drop_duplicates(subset=["채무자키"], keep='first')
    keys_debt = keys_debt.drop_duplicates(subset=["매각사", "원채권사"], keep='first').채무자키.values
    keys_grt = df_grt[(df_grt.보증인성명인.str.contains(name))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
    keys = np.append(keys_debt, keys_grt)
    keys_set = set(keys)
    
    if len(keys) == 0 : 
        os.rename(src, join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
    elif len(keys) ==1 :
        new_name = str(keys[0]) + " " + stem + add_file_name + ext
        try : 
            os.rename(src, join(path_completed, new_name))
            success.append([path, f, path_completed, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
    else : 
        os.rename(src, join(path_multi, f))
        multi.append([path, f, keys_set])

write_log_csv_upload(success, path_success, addInfo)
write_log_csv_upload(noDB, path_noDB, "noDB")
write_log_csv_upload(fail, path, "fail")
write_log_csv_upload(multi, path, "multi")

loss = total - (len(success) - (copy_cnt - conpy_origin_cnt)) - len(noDB) - len(fail) - cnt_noexp -len(multi)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success) - (copy_cnt - conpy_origin_cnt)}개, multi수작업 : {len(multi)}, \
파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개, 정규식없음 : {cnt_noexp}, 복사건 : {conpy_origin_cnt, copy_cnt}")

  0%|          | 0/45 [00:00<?, ?it/s]

100%|██████████| 45/45 [00:00<00:00, 167.07it/s]

log 파일이 비어있습니다. path : D:\전산부재\광주 양도통지서스캔 양통종적 양통종적스캔 한울
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\양통종적 스캔(수작업)\한울가람 양통종적(수작업)\1차검색실패(수작업)
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\■양도통지서 스캔 (수작업)\양통종적 (수작업)\양통종적 스캔(수작업)\한울가람 양통종적(수작업)\1차검색실패(수작업)
파일누수 0개
총 파일 : 45, 파일명 변경 : 45개, multi수작업 : 0, 파일명 변경 실패 : 0개, 전산없음 : 0개, 정규식없음 : 0, 복사건 : (0, 0)


In [25]:
df_grt

,채무자키,매각사구분,보증인성명인,주민번호인,계좌번호,채무자관계,채무자명,보증구분,보증인상태,보증금액,...,보증일,종결일,종결사유,관할법원,사건번호,보증인키,계좌키,원채권사,매각사,채무자주민번호
0,20412714,DNP-02,정희원,580824-1057914,5052-16-10081863,기타,㈜원숭이학교 대표 정희원,포괄근보증,정상,338000000,...,,,,,,200183228,200919441,국민은행,디엔피에이엠씨대부㈜,404-81-13967
1,20412714,DNP-02,정희원,580824-1057914,5052-16-10082013,기타,㈜원숭이학교 대표 정희원,특정근보증,정상,130000000,...,,,,,,200183229,200919442,국민은행,디엔피에이엠씨대부㈜,404-81-13967
2,20412715,DNP-01,홍석기,600831-1143613,2424-16-04001914,기타,㈜브라이튼테크노,포괄근보증,정상,,...,,,,,,200183230,200919452,국민은행,디엔피에이엠씨대부㈜,126-81-62438
3,20412718,DNP-02,김순자,460605-2093911,KEB-C-03288-L-01,보증인,최승필,포괄근보증,사망,5530000,...,,,,,,200183231,200919462,외환카드,디엔피에이엠씨대부㈜,700726-1******
4,20412719,DNP-01,윤순님,560407-2551811,KEB-C-03341-L-01,보증인,김해성,포괄근보증,파산(신청중),8430000,...,,,,,,200183232,200919463,외환카드,디엔피에이엠씨대부㈜,530711-1******
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,20495750,전북-01,고명란,660215-2405919,0001-0710-1149-6278,,동양전자㈜,,정상,,...,,,,,,200198754,201018889,㈜전북은행,㈜전북은행,312-81-87272
10298,20431042,KB카드-01,정영모,570503-1543418,2020500318867,,정영선,,종결,,...,,2022-12-05,보증면탈,,,200198845,200951854,KB국민카드,KB국민카드,641005-2******
10299,20494390,전북-01,김윤식,570308-1543511,0000-7117-3000-1595,,백남연,,정상,,...,,,,,,200199523,201017529,㈜전북은행,㈜전북은행,411223-2******
10300,20494391,전북-01,김윤식,570308-1543511,0000-7117-6000-4465,,백남연,,정상,,...,,,,,,200199524,201017530,㈜전북은행,㈜전북은행,411223-2******


In [ ]:
df_c.columns

Index(['채무자키', '매각사구분', '채권구분', '성명', '주민번호인', '담당자', '채무상태', '성별', '연령',
       '최초원금', '최초합계', '관리자기타', '비고', '보증인성명', '보증인상태', '원채권키', '원채권사',
       '타채무자키'],
      dtype='object')

In [ ]:
won = "김제선"
df_c[df_c.성명.str.contains(won)]

,채무자키,매각사구분,채권구분,성명,주민번호인,담당자,채무상태,성별,연령,최초원금,최초합계,관리자기타,비고,보증인성명,보증인상태,원채권키,원채권사,타채무자키
15461,20458931,KCR,CCRS(프리),김제선,681202-2785429,종결,종결,여자,53,7444628,7781136,9005000893,,,,0,롯데카드,9005000893


In [ ]:
f = "김병의의보증인123456.pdf"
#p_space = re.compile("(?<!\s)의(?=(보|연|대|상|피|))")
p_space = re.compile("(?<!\s)의\s?(?=[보연대상피])|(?<=[가-힣])(?=\()|망\s")
p_space.sub(" ", f)

'김병의 보증인123456.pdf'

In [ ]:
# 전처리 *양통 기준

path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\6.(9005) KCR_헬프\2.원인서류(대출신청서)\롯데서류 (일련번호 변경x)"

path_etc = join(path, "타문서")
path_trace = join(path, "종적")
if not os.path.exists(path_etc) : os.mkdir(path_etc)
if not os.path.exists(path_trace) : os.mkdir(path_trace)

file_list = file_listFnc(path)
total = len(file_list)
success, fail = [], []
noChange_cnt = 0

# 신용의 경우 확인 필요!!
p_etc = re.compile("안내|원인|입회|녹취|초본|등본|등기|등록|승계|집행|판결|결정|매매|대위|분할|분납|상환|감면|신청|통지|통보|예고|표시|고지|위임|채무|종결|회생|파산|신복|신용|압류|타채|대장|등록|은행|원리?금|부채|재산|화해|권고|경매|유체|배당|내용")
p_trace = re.compile(r"종[적족]\s?(조회)?|(?<!주소|소\s)이력")
p_sub = re.compile("양통|양(도|토)\s?통지서?|\(?한울\s?(가람)?\)?|\(?[주유]\)|\([주유]\)?")
p_space = re.compile("(?<!\s)의\s?(?=[보연대상피])|(?<=[가-힣])(?=\()|망\s")


for f in tqdm(file_list, total=total) :
    new_name = f
    new_name = p_sub.sub("", new_name)
    new_name = p_space.sub(" ", new_name)

    if p_etc.search(new_name) :
        try : 
            os.rename(join(path, f), join(path_etc, new_name))
            success.append([path, f, new_name, path_etc, "타문서"])
        except Exception as e :
            fail.append([path, f, new_name, path_etc, e.__class__])
            pass
    elif p_trace.search(new_name) :
        try :
            os.rename(join(path, f), join(path_trace, new_name))
            success.append([path, f, new_name, path_trace, "종적"])
        except Exception as e :
            fail.append([path, f, new_name, path_trace, e.__class__])
            pass
    else :
        if new_name != f :
            try :
                success.append(re_name(join(path, f), join(path, new_name))) #### 같은 이름 충분히 나올 수 있다.
            except Exception as e :
                fail.append([path, f, new_name, path, e.__class__])
                pass
        else :
            noChange_cnt += 1

write_log_csv(success, path, "success")
write_log_csv(fail, path, "fail")
print(f"{total-len(success)-len(fail)-noChange_cnt}개 파일 누수")

100%|██████████| 1452/1452 [00:00<00:00, 104183.14it/s]

log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\3.채권양도통지서\펜타양통
0개 파일 누수


In [ ]:
# 보증인 / 전산삭제된 사람인데, 동명이인이 채무자로 한 명 있어서 여기에 걸린 경우의 오류가 있을 수 있다.(대표로 4~50개를 확인하고 있지만, 확률이 낮은 만큼 전체검토를 하지 않는 이상 확신하기 어렵다.)
###########################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\6.(9005) KCR_헬프\2.원인서류(대출신청서)\! 2620 건 추출완료 (파일이동예정)"
addInfo = "서울 채권별 6.KCR 2.원인 롯데 추출"
add_file_name = "" # 공백주의
# path_noDB = join(r'D:\전산부재', addInfo)
path_noDB = join(path, "결과없음")
path_multi = join(path, "multi")
path_grt = join(path, "보증인")
path_grt_rgt = join(path_grt, "rgt")
path_completed = join(r"D:\업로드완료", addInfo)
###########################################

if not os.path.exists(path_completed) :
    os.makedirs(path_completed)
if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)
if not os.path.exists(path_multi) :
    os.makedirs(path_multi)
if not os.path.exists(path_grt) :
    os.makedirs(path_grt)
if not os.path.exists(path_grt_rgt) :
    os.makedirs(path_grt_rgt)

p_name = re.compile("[가-힣]{2,}")
p_rgt = re.compile("[\d\-]{10,}") ######################

#log
# 검색 결과 여러건인경우
multi = []
noDB = []
success = []
fail = []
grt_cnt = 0

file_list = file_listFnc(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    name, rgt, grt = "없음", "없음", "없음" # 빈문자열이면 검색되어버리니 초기값 설정
    stem, ext = os.path.splitext(f)

    df_result_all = None
    if p_name.search(stem) :
        name = p_name.search(stem).group().strip() # 채무자 이름

        if p_rgt.search(stem) and (re.search("보증|연대|상속|청산", stem) == None): # 채무자이고 rgt는 있는 경우 반드시 일치해야(단, 보증인 등일 경우 rgt쓰지 않는다), grt는 전산에서 사라질 수 있으니 multi일 경우에만 보조적으로
            rgt = p_rgt.search(stem).group()
            df_result_all = df_c[(df_c.성명.str.contains(name)) & (df_c.주민번호인.str.match(rgt))]
        else : 
            df_result_all = df_c[(df_c.성명.str.contains(name))]
            
    else :
        print(f, "패턴 없음")
        continue

    df_result = df_result_all.drop_duplicates(subset=["채무자키"], keep='first')
    keys = df_result.채무자키.values # 시리즈, 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로

    if len(keys) == 0 : 
        # 채무자만 대상으로 할경우 주석처리 [시작]-------------------
        # if len(p_name.findall(stem)) == 1 : # 이름이 하나인데 채무자가 아니라면 보증인에서 검색 (사후 확인 필요)
        #     if rgt != "없음" :
        #         keys = df_grt[(df_grt.보증인성명인.str.contains(name)) & (df_grt.주민번호인.str.match(rgt))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
        #         path_grt_which = path_grt_rgt
        #     else :
        #         keys = df_c[(df_c.보증인성명.str.contains(name))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
        #         path_grt_which = path_grt

        #     if len(keys) == 1 :
        #         new_name = str(keys[0]) + " " + stem + add_file_name + ext #####
        #         try : 
        #             os.rename(join(path, f), join(path_grt_which, new_name))
        #             success.append([path, f, new_name, path_grt_which, "보증인"])
        #             grt_cnt += 1
        #             continue
        #         except Exception as e :
        #             fail.append([f, new_name, e.__class__, e])
        #             continue
        # 채무자만 대상으로 할경우 주석처리 [끝]-------------------
                
        # p_name 이 여러개이거나, 하나이지만 key값을 얻지 못한 경우
        os.rename(join(path, f), join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])

    elif len(keys) ==1 : # 이름만 본 것이기 때문에 오타 또는 동명이인 가능성도 배재할 수는 없음. 모두 검토는 어려우니 4~50개 검토
        new_name = str(keys[0]) + " " + stem + add_file_name + ext #####
        try : 
            os.rename(join(path, f), join(path_completed, new_name))
            success.append([path, f, new_name, path_completed])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])
            continue
    else : # 여러 건
        is_grt = re.search("보증인", stem) # '보증인'이라는 말이 있기 때문에 별도 확인 필요 없음 > path_completed로, grt_cnt도 하지 않는다.
        if is_grt :
            ind = is_grt.end()
            try :
                grt = p_name.search(stem[ind:]).group()
            except : 
                os.rename(join(path, f), join(path_multi, f))
                multi.append([path, f, path_multi, f])
                continue

            keys = df_result[df_result.보증인성명.str.contains(grt)].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
            if len(keys) == 1 : # 한울가람의 경우 1/3이 여기서 처리됐다 ^^
                new_name = str(keys[0]) + " " + stem + add_file_name + ext #####
                try : 
                    os.rename(join(path, f), join(path, new_name))
                    success.append([path, f, new_name, path_completed, "채무자+보증인"])
                    continue
                except Exception as e :
                    fail.append([f, new_name, e.__class__, e])
                    continue
        
        # 보증인 넣어니 0건/여러건이거나, 보증인 없는 경우
        os.rename(join(path, f), join(path_multi, f))
        multi.append([path, f, path_multi, f])

loss = total - len(success) - len(multi) - len(fail) - len(noDB)
print(f"파일누수 {loss}개, 보증인 {grt_cnt}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 결과 여러건 : {len(multi)}개, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개")

write_log_csv(success, path_success, addInfo)
write_log_csv(noDB, path_noDB, "결과없음")
write_log_csv(fail, path, "fail")
write_log_csv(multi, path_multi, "multi")

100%|██████████| 2398/2398 [00:25<00:00, 92.91it/s] 

파일누수 0개, 보증인 0개
총 파일 : 2398, 파일명 변경 : 2023개, 결과 여러건 : 373개, 파일명 변경 실패 : 0개, 전산없음 : 2개
log 파일이 비어있습니다. path : D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\6.(9005) KCR_헬프\2.원인서류(대출신청서)\! 2620 건 추출완료 (파일이동예정)


##### 보증인 이름만

In [ ]:
df_grt.columns

Index(['채무자키', '매각사구분', '보증인성명인', '주민번호인', '계좌번호', '채무자관계', '채무자명', '보증구분',
       '보증인상태', '보증금액', '보증잔액'],
      dtype='object')

In [ ]:
df_grt[df_grt.보증잔액.str.contains("2876031")]

,채무자키,매각사구분,보증인성명인,주민번호인,계좌번호,채무자관계,채무자명,보증구분,보증인상태,보증금액,보증잔액


In [ ]:
# 검색결과 하나인 경우만 처리하고 나머지는 그냥 그대로 둠. 처리 안 된 건 주 채무자 이름 넣고 윗 셀로 ~~
# 보증인 참조로 하자.
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울(전체)양도통지서(헬프매각) - 일부 양통 있음\1803_위드고양통 (양통 있음)\위드고보증인양통\분리"
addInfo = "광주 통합 2.한울 한울(전체) 1803 보증인"
add_file_name = " 양통 위드고 한울" # 공백주의
path_grt = join(path, "보증인")
###########################################

if not os.path.exists(path_grt) :
    os.makedirs(path_grt)

p_name = re.compile("[가-힣]{2,}")
p_rgt = re.compile("\d{2,}")

success = []
fail = []

file_list = file_listFnc(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    name, rgt, grt = "없음", "없음", "없음" # 빈문자열이면 검색되어버리니 초기값 설정
    stem, ext = os.path.splitext(f)
    df_result_all = None

    if p_name.search(stem) :
        name = p_name.search(f).group()
        keys = df_c[(df_c.보증인성명.str.contains(name))].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
        if len(keys) == 1 :
            new_name = str(keys[0]) + " " + stem + add_file_name + ext #####
            try : 
                os.rename(join(path, f), join(path_grt, new_name))
                success.append([path, f, new_name, path_grt, "보증인"])
                continue
            except Exception as e :
                fail.append([f, new_name, e.__class__, e])
                continue

loss = total - len(success) - len(fail)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 결과 여러건 : {len(multi)}개, 파일명 변경 실패 : {len(fail)}개, 전산없음 : {len(noDB)}개")

write_log_csv(success, path_success, addInfo)
write_log_csv(fail, path, "fail")

100%|██████████| 81/81 [00:00<00:00, 587.75it/s]

파일누수 20개
총 파일 : 81, 파일명 변경 : 61개, 결과 여러건 : 0개, 파일명 변경 실패 : 0개, 전산없음 : 0개
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울(전체)양도통지서(헬프매각) - 일부 양통 있음\1803_위드고양통 (양통 있음)\위드고보증인양통\분리


In [ ]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\모음\정리할것\중복확인필요\중복건확인필요\티와이머니1차 양통\1차검색실패(수작업)"
addInfo = "광주 모음 정리할것 중복확인 티와이머니1차"
add_file_name = " 양통 TYM DNP " # 앞뒤 한칸 필수 
# seller = r"한울가람"
path_noDB = join(r'D:\전산부재', addInfo)
path_noExcel = join(path_noDB, "noExcel")
###########################################

# path_completed = join(r"D:\업로드완료", addInfo)
if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)
if not os.path.exists(path_noExcel) :
    os.makedirs(path_noExcel)

p_name = re.compile("[^()][A-Z가-힣]+[^())]") # 앞에 있든 뒤에 있든 (주)빼고

#log
# 검색 결과 여러건인경우
# df_name_multi_result = pd.DataFrame(None, columns=df1.columns)
noDB = []
success = []
noExcel = []
# multi_cnt = 0

file_list = file_listFnc(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    name = "없음" # 빈문자열이면 검색되어버리니 초기값 설정
    stem, ext = os.path.splitext(f)
    # stem_removed = re.sub(r"의?\s?(보증인|상속인)", " ", stem) # 반드시 공백으로 대체 
    # stem_removed = re.sub(r"\([\d주]\)", "", stem_removed) # (주) (1) 제거
    if p_name.search(stem) :
        name = p_name.search(stem).group().strip() # 채무자 이름
        # stem_removed = stem_removed.replace(name, "") # 채무자 이름 제거 (주) 때문에 re쓰지 않고 replace 사용
        
        # if p_name.search(stem_removed) :
        #     grt_name = p_name.search(stem_removed).group().strip() # 보증인 이름
        #     grt_name = re.sub("의.*", "", grt_name) # 이름에 의가 들어간 것은 그냥 수작업하자. 일단 이 폴더는 양통이라 보증인 한명이다. 
    else :
        os.rename(join(path, f), join(path, "nocompile", f))
        continue

    df_result_all = df1[(df1.차주명.str.contains(name)) | (df1.파일명.str.contains(name))]
    df_result = df_result_all.drop_duplicates(subset=["매각 차주 번호"], keep='first')
    keys = df_result["매각 차주 번호"].values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로

    if len(keys) == 0 : # 보증인 대신 매각사 조건 넣고 결과 하나면 변환
        os.rename(join(path, f), join(path_noExcel, f))
        noExcel.append([path, f, path_noExcel, f])
    else :
        os.rename(join(path, f), join(path_noDB, keys[0]+" "+f))
        noDB.append([path, f, path_noDB, keys[0]+" "+f])

100%|██████████| 1021/1021 [00:06<00:00, 169.59it/s]


In [ ]:
p_name.search("강영실")

<re.Match object; span=(0, 3), match='강영실'>

##### 양통 종적 : 엑셀list 있는 경우
- 이름 > 일련번호

In [ ]:
path_excel = r"D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\KS자산관리 양통\양통종적조회(KS)(2).xlsx"
df_trace = pd.read_excel(path_excel)
#df_trace = df_trace[["성명", "일련번호", "등기번호", "주민번호"]]
df_trace = df_trace.astype(dtype='str')
# 성명 > 일련번호 , 참고 등기번호, 주민번호

In [ ]:
###########################################
path = r"D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\KS자산관리 양통\양통 종적"
addInfo = r"광주 통합 6.KS KS양통 종적"
add_file_name = ""
###########################################

# path_noDB = join(r'D:\전산부재', addInfo)
# path_completed = join(r"D:\업로드완료", addInfo)
# if not os.path.exists(path_noDB) :
#     os.makedirs(path_noDB)
# if not os.path.exists(path_completed) :
#     os.makedirs(path_completed)

p_name = re.compile("[가-힣]{2,}") 
# log
# 검색 결과 여러건인경우
# df_name_multi_result = pd.DataFrame(None, columns=df_trace.columns)
success = []
fail = []
errExcel_cnt = 0

file_list = file_listFnc(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    name = p_name.search(stem).group()
    
    df_result = df_trace[df_trace.성명.str.contains(name)]
    sr_rgt = df_result.일련번호.values # 생년월일은 시작이니 match(전체 할 경우 df_c에 하이픈 있음에 유의), 성명은 '변경전'같은 거 있으니 포함여부로

    if len(sr_rgt) ==1 :
        rgt = re.sub("[^0-9\-]", "", sr_rgt[0])
        new_name = rgt + " " + stem + add_file_name + ext #############
        try : 
            os.rename(join(path, f), join(path, new_name))
            success.append([f, new_name])
        except Exception as e :
            fail.append([f, new_name, e.__class__, e])

    else : 
        errExcel_cnt += 1

loss = total - len(success)- len(fail) - errExcel_cnt
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 파일명 변경 실패 : {len(fail)}개, 엑셀검색실패 : {errExcel_cnt}개")

write_log_csv(success, path_success, addInfo+"일련번호추가")
write_log_csv(fail, path, "fail")

100%|██████████| 51/51 [00:00<00:00, 106.19it/s]

파일누수 2개
총 파일 : 51, 파일명 변경 : 49개, 파일명 변경 실패 : 0개
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\KS자산관리 양통\양통 종적


In [ ]:
name = "박일규"
df_trace[df_trace.성명 == name]

,성명,일련번호,등기번호,주민번호


In [94]:
df_debt[df_debt["원채권사"].str.contains("해동신용")]

,채무자키,매각사구분,채권구분,성명,주민번호인,담당자,채무상태,성별,연령,원채권사,매각사,최초원금,최초합계,현재원금,관리자기타,비고,종결일,보증인성명,보증인상태,타채무자키


In [96]:
event = "2003가소1222"
keys = df_law[(df_law.사건번호==event)|(df_law.사건번호==event[2:])|(df_law.관련사건번호==event)|(df_law.관련사건번호==event[2:])].drop_duplicates(subset=["채무자키"], keep='first').채무자키.values
keys

array(['20419990'], dtype=object)

##### 단순 이름 변경 / 이동

In [12]:
#################################################
path = r"D:\구 스캔파일\3.채권양도통지서\1차 양통\보증인\새 폴더"
check = "양통"
addInfo = ""
out_dir = path # join(path, "사건번호있음")
if not os.path.exists(out_dir) : os.mkdir(out_dir)
#################################################
success , fail = [], []
file_list = read_filelist(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    new_name = f
    if re.search(check, new_name) :
        #new_name = os.path.splitext(f)[0] + " 양통" + os.path.splitext(f)[1]
        #key = re.search("\d{8}", f).group()
        #new_name = key + " " + f
        new_name = re.sub(check, "양통 ", new_name)
    
        try :
            os.rename(join(path, f), join(out_dir, new_name))
            success.append([path, f, out_dir, new_name])
        except Exception as e :
            fail.append([path, f, e.__class__])
            continue

print(f"{total}개 중 {len(success)}개 파일명 변경, {len(fail)}개 실패")
# write_log_csv(success, path_success, addInfo)

100%|██████████| 412/412 [00:00<00:00, 4806.72it/s]

412개 중 412개 파일명 변경, 0개 실패


##### 폴더명 바꾸기

In [76]:
####################################
path = r"D:\카카오\법원문건(개인회생)"
addInfo = "카카오 법원문건(개인회생)"
path_noDB = join(r'D:\전산부재', addInfo)
####################################
dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total = len(dir_list)
success, fail, noDB = [], [], []

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)

for dir in tqdm(dir_list, total= total) :
    try :
        new_dir = dict[dir]
    except :
        os.rename(join(path, dir), join(path_noDB, dir))
        noDB.append([path, dir, noDB, dir])
        continue
    try :
        os.rename(join(path, dir), join(path, new_dir))
        success.append([path, dir, path, new_dir])
    except Exception as e :
        fail.append([path, dir, path, new_dir, e.__class__])
        continue

write_log_csv(success, path_success, "카카오 법원문건 폴더명바꾸기")
write_log_csv(fail, path, "fail")
write_log_csv(noDB, path_noDB, "전산없음")

print(f"누수 : {total - len(success) - len(fail) - len(noDB)}")
print(f"전체 : {total}, 성공 : {len(success)}, 실패 : {len(fail)}, 전산없음 : {len(noDB)}")

100%|██████████| 1330/1330 [00:00<00:00, 3753.86it/s]

누수 : -6
전체 : 1330, 성공 : 1324, 실패 : 6, 전산없음 : 6


##### 하위폴더 만들기(분류하기)

In [4]:
path = r'\\192.168.0.75\데이터관리팀\박준혁 전병만\원인서류 71추가'
file_list = read_filelist(path)
total = len(file_list)
success, fail = [], []

for f in tqdm(file_list, total=total) :
    dir = f[:8]  ###################
    dst_dir = join(path, dir)
    if not os.path.exists(dst_dir) :
        os.mkdir(dst_dir)
    try :
        os.rename(join(path, f), join(dst_dir, f))    
        success.append([path, f, dst_dir, f])
    except Exception as e :
        fail.append([path, f, dst_dir, f, e.__class__])
        continue
    
print(f"파일누수 : {total - len(success) - len(fail)}")
print(f"전체 : {total}, 성공 : {len(success)}, 실패 : {len(fail)}")

100%|██████████| 70/70 [00:12<00:00,  5.67it/s]

파일누수 : 0
전체 : 70, 성공 : 70, 실패 : 0


##### 상위 폴더로 파일 끄집어내기

In [64]:
path = r'D:\구 스캔파일\광주 공유폴더\■집행문,판결문\기존문서\판결문'
addInfo = "광주 집행문판결문"
add_file_name = "" # 공백 유의

success, fail = [], []
for root, dirs, files in tqdm(os.walk(path)) :
    #if os.path.basename == "0001" : 

    for f in files :
        stem, ext = os.path.splitext(f)
        #if re.search("pdf|PDF", ext) : ####
        try :
            # ## 상위 폴더명을 파일이름에 붙여 꺼내기
            # dir = os.path.basename(root)[:8] ############
            # new_stem = dir + " " + stem + add_file_name
            # new_name = new_stem + ext
            
            ## 그냥 꺼내기
            new_stem = stem
            new_name = new_stem + ext

            sn = 0
            while os.path.exists(join(path, new_name)) :
                sn += 1 
                new_name = new_stem + " " + str(sn) + ext
            os.rename(join(root, f), join(path, new_name)) # os.path.dirname(path)
            success.append([root, f, path, new_name])

        except Exception as e:
            fail.append([root, f, e.__class__, e])
            pass

print(f"{len(success)}건 파일 이동 완료, {len(fail)}건 파일 이동 실패")
ff.write_log_csv_upload(success, path_success, addInfo)
ff.write_log_csv_upload(fail, path, "fail")

19it [01:18,  4.15s/it]


6003건 파일 이동 완료, 0건 파일 이동 실패
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\■집행문,판결문\기존문서\판결문


##### 빈 폴더 지우기

In [2]:
path = r'D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서'
cnt = 0
for root, dirs, files in tqdm(os.walk(path)) :
	# 내for문 없이 root바로 remove해도 되는데, 이렇게 하면 수가 조금 줄어드네.
        for dir in dirs :
            try :
                os.removedirs(os.path.join(root, dir))
                cnt += 1
                print(cnt, os.path.join(root, dir))
                #os.rmdir(os.path.join(root, dir))
            except : # 파일 있는 경우
                # print(root, " ", dir)
                pass
print(f"{cnt}개의 빈 폴더 삭제")

1it [00:00, 51.58it/s]

1 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\10422-10921
2 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\10922-11421
3 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\11422-11921
4 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\11922-12421
5 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\12422-12921
6 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\12922-13421
7 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\13422-13921
8 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\13922-14421
9 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\14422-14921
10 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\14922-15421
11 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\15422-15921
12 D:\구 스캔파일\광주 공유폴더\모음 (소수, 수작업)\대출원장(양통 소수파일만 남음)\양통찿기(T-1)\TF\양도통지서\15922-16421
13 D:\구 스캔파일\

##### 종결건 처리

In [6]:
def out(file_list:list, dict_refer:dict) :
    p_out = re.compile(r'개인회생\(면책\)|파산\(면책\)|환매|매각|종결')
    out_list = []
    noKey_list = []
    for f in tqdm(file_list) :
        key = f[:8]
        try : 
            if p_out.match(dict_refer[key][1]) : 
                out_list.append(f)
        except : 
            noKey_list.append(f)
    return out_list, noKey_list



In [9]:
path = r'D:\구 스캔파일\광주 공유폴더\모음\대출원장\삼성대출원장 스캔'
file_list = file_listFnc(path)
out_list, noKey_list = out(file_list, dict_refer)
(len(file_list), len(out_list), len(noKey_list))

100%|██████████| 216/216 [00:00<?, ?it/s]


(216, 188, 0)

In [8]:
out_list

['20408013 662 이정애.pdf',
 '20411158 485 오원종.pdf',
 '20413589 706 임재완.pdf',
 '20413598 469 양성태.pdf',
 '20413598 470 양성태.pdf',
 '20413607 484 우일건설중기(주).pdf',
 '20413607 484오용배.pdf',
 '20413644 523 위휘종.pdf',
 '20413646 479 오경선.pdf',
 '20455350 475 엄덕남.pdf',
 '20455352 696 임병쳘.pdf',
 '20455354 600 이기호.pdf',
 '20455356 552 유태문.pdf',
 '20455365 465 안병식.pdf',
 '20455367 723 전경일.pdf',
 '20455378 733 전종해 (2).pdf',
 '20455378 733 전종해.pdf',
 '20455381 667 이종창.pdf',
 '20455390 739 전홍식.pdf',
 '20455393 605 이동근.pdf',
 '20455400 524 유관열.pdf',
 '20455405 641 이승호.pdf',
 '20455405 642 이승호.pdf',
 '20455406 679 이택관.pdf',
 '20455411 674 이충로.pdf',
 '20455411 675 이충원.pdf',
 '20455417 661 이정숙.pdf',
 '20455419 487 오형근.pdf',
 '20455424 456 심규현.pdf',
 '20455429 668 이종창.pdf',
 '20455429 669 이종화.pdf',
 '20455431 478 예병순.pdf',
 '20455433 606 이동석.pdf',
 '20455436 557 육명선.pdf',
 '20455438 647 이용호.pdf',
 '20455446 689 임경숙.pdf',
 '20455447 604 이도신.pdf',
 '20455448 718 장미영.pdf',
 '20455449 453 신진운수.pdf',
 '20455453 719 

In [11]:
add_file_name = " 원인서류"
path_out = join(path, "out")
if not os.path.exists(path_out) :
    os.mkdir(path_out)
for f in out_list :
    stem, ext = os.path.splitext(f)
    new_name = stem + add_file_name + ext
    os.rename(join(path, f), join(path_out, new_name))

##### 관리자키 > 채무자키

In [3]:
# 채무자조회에서 원하는 것만 불러와서 dict로 만들기
path_df = r'D:\1.전산,개발\3.파일서버관리\0.월별전산조회\extraKey\dict모음' 
with open(join(path_df, "account.pkl"), "rb") as pkl : dict_account = pickle.load(pkl) # 하이픈 그대로 있음
with open(join(path_df, "admin.pkl"), "rb") as pkl : dict_admin = pickle.load(pkl)
with open(join(path_df, "note.pkl"), "rb") as pkl : dict_note = pickle.load(pkl)
with open(join(path_df, "rgt.pkl"), "rb") as pkl : dict_rgt = pickle.load(pkl) # 하이픈 제거되어 있음

In [3]:
# 현재 만든 pkl은 키와 하나의 value로만 되어있다.(11/17)
# 다른 항목이 필요하다면 엑셀을 df로 불러와서 작업하라.
path_df = r'D:\1.전산,개발\3.파일서버관리\0.월별전산조회\extraKey\dict모음'
with open(join(path_df, "우담sr(106...).pkl"), "rb") as pkl : dict = pickle.load(pkl)
len(dict)

6510

In [4]:
dict["10626211"]

'20459950'

###### 암호 안걸린 엑셀파일 채무자키 연결하기

In [19]:
import warnings
warnings.simplefilter("ignore")
df1 = pd.read_excel(r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\20.(9019) (유)우담자산관리대부_솔림헬프\1.매입전 자료\솔림헬프 매각건_파일번호 - 최종.xlsx")
df2 = pd.read_excel(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\채무자키_계좌번호.xlsx")
df_c = pd.merge(df1, df2, how="inner")
df_c

,순번,채권\n일련번호,계좌번호,성명,매각사,일련번호,약정서\n원본,스캔파일\n약정서,스캔파일\n양도통지서,약정서 원본\n비고,채무자키
0,1,A-1,33061760246755,김예린,상상-04,10801648,X,ㅇ,ㅇ,NaN,20459822
1,2,A-2,33061760230803,도세연,상상-04,10801637,X,ㅇ,ㅇ,NaN,20459823
2,3,A-3,33061760178534,지은비,상상-03,10689877,X,ㅇ,ㅇ,NaN,20459824
3,4,A-4,33061760419237,김도희,상상-04,10801683,X,ㅇ,ㅇ,NaN,20459825
4,5,A-5,33061760334338,안민욱,상상-04,10801677,X,ㅇ,ㅇ,NaN,20459826
...,...,...,...,...,...,...,...,...,...,...,...
6684,6687,A-6687,33061760122698,박아영,상상-02,10654081,X,ㅇ,ㅇ,NaN,20466508
6685,6688,A-6688,459-61-76-0094644,이민서,금융-01,10690427,X,ㅇ,ㅇ,NaN,20466509
6686,6689,A-6689,459-61-76-0086122,박수진,상플-04,10653959,X,ㅇ,ㅇ,NaN,20466510
6687,6690,A-6690,33061760141096,최지애,상상-04,10801553,X,ㅇ,ㅇ,NaN,20466511


In [20]:
df_c = df_c[["일련번호", "채무자키"]]

In [59]:
df_c.to_excel("D:\카카오\채무자키-타채무자키(계좌)-파일명(계좌).xlsx")

In [21]:
df_c = df_c.astype(dtype="str")
df_c.dtypes
df_c.head(2)

,일련번호,채무자키
0,10801648,20459822
1,10801637,20459823


###### 암호걸린 엑셀파일 읽기, 채무자키 연결하기

In [22]:
# 암호걸린 엑셀파일 읽고 DICT 만들기
import xlwings as xw
import numpy as np
book1 = xw.Book(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\TYM_1차.xlsx", password="1234")
df1 = book1.sheets(1).used_range.options(pd.DataFrame).value
#df["일련번호"] = df["일련번호"].apply(np.int64).apply(np.str_)
df1
# df = pd.read_excel(r'D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\TYM.xlsx')
#df.columns = ["구채무자키", "계좌번호"]
#df
# print(len(df.index))
# df.fillna(0, inplace=True)
# print(len(df.index))
# df.head()

,매각 차주 번호,계좌번호,차주명,구분,파일명
폴더 순번,,,,,
1.0,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,티와이(보증인),005704038534400100_김종찬 .tif
1.0,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,티와이(차주),TFA39000-0409-7009.TIF
6.0,TYM-13-1-00058,TYM-13-1-00058-01,(주)대길,티와이(차주),TFA39000-0409-7696.TIF
8.0,TYM-13-1-00116,TYM-13-1-00116-01,이수연,티와이(차주),TFA39000-0409-9186.TIF
9.0,TYM-13-1-00117,TYM-13-1-00117-01,류형열,티와이(차주),TFA39000-0409-9207.TIF
...,...,...,...,...,...
19774.0,TYM-13-1-17211,TYM-13-1-17211-01,백오종,동양(차주),WFI-바07735 .tif
19775.0,TYM-13-1-17258,TYM-13-1-17258-01,주흥길,동양(차주),WFI-자16740 .tif
19776.0,TYM-13-1-17266,TYM-13-1-17266-01,예용해,동양(차주),WFI-아10412 .tif


In [20]:
# 채무자키 연결하기
book2 = xw.Book(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\채무자키_계좌번호.xlsx", password="1234")
df2 = book2.sheets(1).used_range.options(pd.DataFrame).value
# df2["채무자키"] = df2["채무자키"].apply(np.int64).apply(np.str_)
df2.reset_index(inplace=True)
df2

,채무자키,계좌번호
0,20412714,1375-16-01001756
1,20412714,5052-16-10081511
2,20412714,5052-16-10081863
3,20412714,5052-16-10082013
4,20455180,8288-09-01017811
...,...,...
75084,20515493,RA18010401001
75085,20515494,RA18070147001
75086,20515495,RA18080389001
75087,20515496,RA18100152001


In [24]:
df_c = pd.merge(df1, df2, how="inner")
df_c

,매각 차주 번호,계좌번호,차주명,구분,파일명,채무자키
0,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,티와이(보증인),005704038534400100_김종찬 .tif,20414516
1,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,티와이(차주),TFA39000-0409-7009.TIF,20414516
2,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,동양(보증인),KP1-자-00534.tif,20414516
3,TYM-13-1-00002,TYM-13-1-00002-01,(주)전우,동양(차주),KP1-자-00534.tif,20414516
4,TYM-13-1-00058,TYM-13-1-00058-01,(주)대길,티와이(차주),TFA39000-0409-7696.TIF,20414518
...,...,...,...,...,...,...
9126,TYM-13-1-16124,TYM-13-1-16124-01,김희순,동양(차주),BR705-01-0175.TIF,20455049
9127,TYM-13-1-16332,TYM-13-1-16332-01,배재용,동양(차주),BR612-01-0301.TIF,20417746
9128,TYM-13-1-16585,TYM-13-1-16585-01,전대규,동양(차주),BR812-01-0666.TIF,20457873
9129,TYM-13-1-17211,TYM-13-1-17211-01,백오종,동양(차주),WFI-바07735 .tif,20417759


###### df > dict

In [22]:
# 중복값 컨트롤
df_c = df_c.drop_duplicates(subset="일련번호", keep="first")

In [23]:
# df의 일부칼럼을 dict로 1 (중복값 사전에 컨트롤 해야)
df_c.set_index("일련번호", inplace=True)
#df_c.drop('타채무자키', axis=1, inplace=True)
dict = df_c.to_dict() ###### 중복값이 있는 경우 나중것만 남는다. 사전에 컨트롤 필요
dict = dict["채무자키"] # col_name(인덱스 하지 않은 col) 이 따라왔네.(컬럼명 없애고 1차원으로 만들기)
dict

KeyError: '일련번호'

In [ ]:
# df의 일부칼럼을 dict로 2
df_t = df_c.astype(dtype = 'str')
dict = { re.sub("-", "", s2):s1 for s1, s2 in zip(df_t.채무자키, df_t.주민번호인)} ######
dict

###### dict to pkl(파일 저장)

In [26]:
with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\dict모음\우담sr(106...).pkl", mode="wb") as pkl :
    pickle.dump(dict, pkl)

In [19]:
f = "10690205 양도통지서 상플 우담"
p = re.compile(r"\d{8}(?=\b)")
p.search(f)


<re.Match object; span=(0, 8), match='10690205'>

###### 파일명 > dict

In [20]:
##############################################
path = r'D:\구 스캔파일\서울\광주채권\외장기본있던것\★공유업데이트 2021.0712\■양도통지서 스캔\한울(전체)양도통지서(헬프매각)\드림(하나)양통(2015.02.06.)\분할'
addInfo = "서울 광주채권 외장 공유업데이트 양통스캔 한울(전체) 드림"
add_file_name = ""
dict = dict_admin
p = re.compile(r"\d{10}(?=\b)")
path_completed = join(r"D:\업로드완료", addInfo)
path_noDB = join(r'D:\전산부재', addInfo)
##############################################
path_noMatch = join(path, "noMatch") # 이건 아래에 만드는 코드 있음.

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)
if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

success = []
fail = []
noDB = []
cnt_noMatch = 0

file_list = file_listFnc(path)
total=len(file_list)

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    ################
    if p.search(stem) :
        e_key = p.search(stem).group()
    else :
        if not os.path.exists(path_noMatch) : os.mkdir(path_noMatch)
        os.rename(join(path, f), join(path, "일련번호 없음", f))
        cnt_noMatch += 1

    try :
        key = str(dict[e_key])
    except :
        os.rename(join(path, f), join(path_noDB, f))
        noDB.append([path, f, path_noDB, f, "엑셀에 e_Key가 없음"])
        continue
    try :
        new_name = key + " " +stem + add_file_name + ext #############################
        os.rename(join(path, f), join(path_completed, new_name))
        success.append([path, f, new_name])
    except Exception as e :
        fail.append([path, f, new_name, e.__class__, e])
        pass
    
loss = total - len(success) - len(fail) - len(noDB) - cnt_noMatch
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(fail)}개 파일명 변경 실패, {len(noDB)}개 수작업, {cnt_noMatch}개 매칭키없음')

write_log_csv(success, path_success, addInfo)
write_log_csv(fail, path, "fail")
write_log_csv(noDB, path_noDB, "noDB")

100%|██████████| 732/732 [00:00<00:00, 3081.56it/s]

0개 파일 누수
732개 중 727개 파일 이름 변경 완료. 0개 파일명 변경 실패, 5개 수작업, 0개 매칭키없음


In [ ]:
file_list = file_listFnc(path)
for f in file_list :
    key = f[:8]
    stem, ext = os.path.splitext(f)
    name = dict_refer[key][2]
    new_name = stem + " " + name + ext
    os.rename(join(path, f), join(path, new_name))

###### excel > df 로 작업

In [16]:
df_e = pd.read_excel(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\kcr.xlsx")
df_e = df_e.astype(dtype='str')
df_e.head(1)

,Unnamed: 0,채무자키,매각사구분,채권구분,성명,일련번호,성명r,비교,Unnamed: 8
0,0,20424102,KCR,CCRS(프리),유온규,B-00001,유온규,True,nan


In [21]:
# 일련번호 채무자키 완전한 경우의 코드
###########################################
path = r"D:\구 스캔파일\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\6.(9005) KCR_헬프\3.채권양도통지서\양통,종적조회 (매입 전)"
addInfo = "서울 채권별 6.KCR 3.양통 매입전"
add_file_name = "" # 공백주의
p = re.compile("B-\d{5}")
###########################################

path_completed = join(r"D:\업로드완료", addInfo)
if not os.path.exists(path_completed) : os.mkdir(path_completed)

success = []
fail = []

file_list = file_listFnc(path)
total = len(file_list)

for f in tqdm(file_list, total=total) :
    
    stem, ext = os.path.splitext(f)
    df_result_all = None

    if p.search(stem) :
        sr = p.search(stem).group()
        keys = df_e[df_e.일련번호 == sr].채무자키.values #############################
        if len(keys) == 1 :
            key = keys[0]
            new_name = key + " " + stem + add_file_name + ext
            try : 
                os.rename(join(path, f), join(path_completed, new_name))
                success.append([path, f, new_name, path_completed])
            except Exception as e :
                fail.append([f, new_name, e.__class__, e])
                continue
        else : fail.append([f, "매칭되는 key가 한 개가 아님"])
    else :
        print("정규표현없음", f)

loss = total - len(success) - len(fail)
print(f"파일누수 {loss}개")
print(f"총 파일 : {total}, 파일명 변경 : {len(success)}개, 파일명 변경 실패 : {len(fail)}개")

write_log_csv(success, path_success, addInfo)
write_log_csv(fail, path, "fail")

100%|██████████| 5069/5069 [00:03<00:00, 1363.78it/s]

파일누수 0개
총 파일 : 5069, 파일명 변경 : 5067개, 파일명 변경 실패 : 2개


###### 파일명 > 일련번호(dict) > 채무자키(dict_refer)

In [6]:
##############################################
path = r'D:\구 스캔파일\광주 공유폴더\■집행문,판결문\승계완료, 판결문\서울-광주이관 승계집행문--변환가능'
addInfo = "광주 집행문판결문 승계완료판결문 서울이관"
dict_ref = dict_admin
add_file_name = ""
##############################################

path_noDB = join(r'D:\전산부재', addInfo)
path_noExcel = join(path_noDB, "noExcel")
path_completed = path #join(r"D:\업로드완료", addInfo)

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)
if not os.path.exists(path_noExcel) :
    os.makedirs(path_noExcel)
if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

noDB = []
noExcel = []
success = []
fail = []

# p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
# file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
file_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(file_list)
# p = re.compile("-(1)$") # 뒤에 꼬리표 떼기. 
# p = re.compile("\D*")
# p0 = re.compile("^0*")
for f in tqdm(file_list, total=total) :
    #stem, ext = os.path.splitext(f)
    # n = stem.strip()
    # n = p0.sub("", n)
    # n = p.sub("", n)
    # n = re.sub("000$", "100", n)
    # nStem = p.sub("", stem)
    # nStem = nStem.strip()
    # nExt = ext.upper() # 대문자 변경
    # n = nStem + nExt
    # name, param = "", ""
    nStem = f
    try :
        sn = dict[nStem]  ############
        # if re.search("[가-힣]+", dict["보증인"]) :
        #     name = re.search("[가-힣]+", dict["보증인"]).group()
        # else :
        #     name = "보증인"
    except :
        os.rename(join(path, f), join(path_noExcel, f))
        noExcel.append([path, f, "noExcel"])
        continue

    try :
        #new_name = sn +" "+ stem + ext
        new_name = sn
        key = dict_ref[sn]  #############
    except :
        os.rename(join(path, f), join(path_noDB, new_name))
        noDB.append([path, f, new_name, "noDB"])
        continue

    try :
        #new_name = key + add_file_name + stem + ext # -1이면 중복이 있다는 소리니 새파일명은 원래 stem을 사용하자
        new_name = key
        os.rename(join(path, f), join(path_completed, new_name))
        success.append([path, f, path_completed, new_name])
    except Exception as e :
        fail.append([path, f, new_name, e.__class__, e])
        continue
    
loss = total - len(success) - len(noDB) - len(noExcel) - len(fail)
print(f"{loss}개 파일 누수")
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(noDB)}개 전산없음, {len(noExcel)}개 엑셀없음, {len(fail)}개 파일명변경실패')
write_log_csv(success, path_success, addInfo)
write_log_csv(noDB, path_noDB)
write_log_csv(noExcel, path_noExcel)
write_log_csv(fail, path)

0it [00:00, ?it/s]

0개 파일 누수
0개 중 0개 파일 이름 변경 완료. 0개 전산없음, 0개 엑셀없음, 0개 파일명변경실패


NameError: name 'write_log_csv' is not defined

###### 파일명 > 일련번호-사업자/주민번호(dict) > 채무자키(refer)

In [28]:
##### 일련번호 > 사업자/주민번호 > 채무자키
## 사업자 번호는 앞뒤에 0이 붙어있어서 없다고 noDB로 나올 수 있음. 
##############################################
path = r'D:\extraKey\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울가람양도통지서\한울가람 양통\광주이관 채권양도통지서\보증인양통'
addInfo = "광주 통합 2.한울 한울양도 양통 광주이관 보증인"
noWhere = join(r'D:\전산부재', addInfo)
dict_ref = dict_rgt
add = "보증인 양통"
##############################################
# p_rgt1 = re.compile(r'((?<=\D)|\b)\d{13}((?=\D)|\b)') # 13자리
# p_rgt2 = re.compile(r'((?<=\D)|\b)(\d{3})-?(\d{2})-?(\d{5})((?=\D)|\b)') # 10자리
no_key = []
success = []
fail = []
noWhere_cnt = 0

file_list = file_listFnc(path)
total=len(file_list)

print(f'{total}개의 파일에 대해 작업 시작')

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)
    sn = stem[:10]
    
    try :
        e_key = dict[sn]
    except : # 키가 참고 엑셀에 없음
        no_key.append([f, "엑셀에 sn없음"])
        continue

    try :
        key = dict_ref[e_key] 
    except :
        os.rename(join(path, f), join(noWhere, f))
        noWhere_cnt += 1
        continue

    try :
        new_name = key + " " + stem + add + ext
        os.rename(join(path, f), join(path, new_name))
        success.append([path, f, path, new_name])
    except Exception as e :
        fail.append([path, f, e.__class__, e])
        continue

loss = total - len(success) - len(no_key) - noWhere_cnt - len(fail)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(no_key)}개 엑셀에 없음, {noWhere_cnt}개 전산없음, {len(fail)}개 파일변경중 에러발생')

write_log_csv(success, path_log, addInfo)
write_log_csv(no_key, path, "엑셀에없음")
write_log_csv(fail, path, "fail")

604개의 파일에 대해 작업 시작


100%|██████████| 604/604 [00:00<00:00, 5728.85it/s]

0개 파일 누수
604개 중 590개 파일 이름 변경 완료. 14개 엑셀에 없음, 0개 전산없음, 0개 파일변경중 에러발생
log 파일이 비어있습니다. path : D:\extraKey\광주 공유폴더\통합파일\(2.한울가람_헬프)\한울가람양도통지서\한울가람 양통\광주이관 채권양도통지서\보증인양통


###### 사업자/주민번호 > 채무자키

In [29]:
## 사업자 번호는 앞뒤에 0이 붙어있어서 없다고 noDB로 나올 수 있음. 
#  윗셀이랑 혼합되서 원래 코드 지워져버림.



##############################################
path = r'D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = "광주 모음 대출원장 케이에스 펜타"
noWhere = join(r'D:\전산부재', addInfo)
dict_ref = dict_rgt
add = ""
##############################################
p_rgt1 = re.compile(r'((?<=\D)|\b)\d{13}((?=\D)|\b)') # 13자리
p_rgt2 = re.compile(r'((?<=\D)|\b)(\d{3})-?(\d{2})-?(\d{5})((?=\D)|\b)') # 10자리
no_key = []

p_extension = re.compile('(jpeg|jpg|bmp|gif|pdf|png|tif)$', re.I)
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0

print(f'{total}개의 파일에 대해 작업 시작')

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)

    try :
        key = dict_ref[e_key]
    except :
        no_key.append([f, "noDictKey"]) # 전산에 없음
        pass
    try :
        new_name = key + " " + n + add + ext
        os.rename(join(path, f), join(path, new_name))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        pass
loss = total - cnt
print(f'{total}개 중 {cnt}개 파일 이름 변경 완료. {loss}개 확인 요망')
write_log_csv(no_key, path)

7381개의 파일에 대해 작업 시작


  1%|          | 85/7381 [00:00<00:08, 846.26it/s]

6001000022 M2_5708111530315_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 M2_5708111530315_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000022 N2_3901262530326_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 N2_3901262530326_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000022 N2_6210112530312_TYM_PENCO_001.pdf : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000022 N2_6210112530312_TYM_PENCO_001.pdf' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.pdf'
6001000084 N2_7609172067438_

 13%|█▎        | 930/7381 [00:00<00:01, 3402.64it/s]

6001000751 06813_2002_444_B_09_000000000000000_12.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_12.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001000751 06813_2002_444_B_09_000000000000000_2.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_2.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001000751 06813_2002_444_B_09_000000000000000_3.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001000751 06813_2002_444_B_09_000000000000000_3.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC

...

[WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_4_08_000000000000000_3.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_4_08_000000000000000_4.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_4_08_000000000000000_4.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_B_14_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001327 06845_2002_339_B_14_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001327 06845_2002_339_B_99_000000000000000_1.tif : <class 'FileEx

 86%|████████▌ | 6343/7381 [00:01<00:00, 3675.35it/s]

6001001442 02445_2002_748_4_08_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001442 02445_2002_748_4_08_000000000000000_10.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_10.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001442 02445_2002_748_4_08_000000000000000_11.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001442 02445_2002_748_4_08_000000000000000_11.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_B

100%|██████████| 7381/7381 [00:02<00:00, 3622.20it/s]

6001001702 08590_2002_9414_D_01_000000000000000_7.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001702 08590_2002_9414_D_01_000000000000000_7.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001702 08590_2002_9414_D_01_000000000000000_8.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001702 08590_2002_9414_D_01_000000000000000_8.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_6503251465213_BSC_PENCO_002.tif'
6001001705 02445_2002_10218_4_99_000000000000000_1.tif : <class 'FileExistsError'> : [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\6001001705 02445_2002_10218_4_99_000000000000000_1.tif' -> 'D:\\extraKey\\광주 공유폴더\\모음\\대출원장\\케이에스자산관리\\펜타원인서류\\20429422 9001001124 M1_65032514652

###### 주민번호 > 채무자키(낱장처리)

In [18]:
# 폴더명인 계좌번호가 엑셀에 없어서 파일명에 있는 주민번호로 다시 한 번 확인한 뒤 조치하는 코드
path = r'D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\11.(9010) 하나캐피탈_헬프\2.원인서류(대출신청서)'
path_origin = r"D:\extraKey\서울\! 채권별 스캔본 (한울가람, 9001~)\! 채권별 스캔본 (한울가람, 9001~)(스캔파일)\1.한울가람_헬프\! 매입전자료(한울가람스캔자료)\원인서류\법인"
addInfo = "서울 채권별 한울가람 매입전 원인서류 법인"
add_file_name = "" # 공백 하나
p_rgt = re.compile(r'((?<=\D)|\b)\d{10}((?=\D)|\b)') 
##############################################
path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
dict_ref = dict_rgt
noDB = []
success = []
fail = []

dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)

if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

for d in tqdm(dir_list, total=total) :
    f = os.listdir(join(path, d))[0]
    if p_rgt.search(f) :
        e_key = p_rgt.search(f).group()
        try :
            key = dict_ref[e_key]
        except :
            os.rename(join(path, d), join(path_noDB, d))
            noDB.append([path, d, path_noDB, d])  #############
            continue
        
        try :
            dst = join(path_origin, key)
            sn = 0
            while os.path.exists(dst) :
                sn += 1 
                dst = join(path_origin, key + " " + str(sn))
            os.rename(join(path, d), dst) ###########
            success.append([path, d, path_origin, key]) 
        except Exception as e: 
            fail.append([path, d, path_origin, key, e.__class__]) ###
            continue

loss = total - len(success) - len(fail) - len(noDB)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(fail)}개 파일 이름 변경 실패. {len(noDB)}개 전산없음')

write_log_csv(success, path_success, addInfo)
write_log_csv(noDB, path_noDB, addInfo)
write_log_csv(fail, path, addInfo)

100%|██████████| 337/337 [00:00<00:00, 15369.26it/s]

0
0


###### 관리자번호

In [93]:
##############################################
path = r'D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\KS자산관리 양통\양통 종적'
addInfo = "광주 통합 6.KS KS양통"
add_file_name = "" # 공백 하나
dict_ref = dict_admin
p_admin = re.compile(r'((?<=\D)|\b)\d{10}((?=\D)|\b)') # 10자리
#dict_ref = dict
#p_admin = re.compile(r'((?<=\D)|\b)\d{8}((?=\D)|\b)') # 8자리
##############################################
path_noDB = join(r'D:\전산부재', addInfo)
path_completed = join(r"D:\업로드완료", addInfo)
noDB = []
success = []
fail = []
file_list = file_listFnc(path)
total=len(file_list)

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)
if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

for f in tqdm(file_list, total=total) :
    stem, ext = os.path.splitext(f)

    if p_admin.search(stem) :
        e_key = p_admin.search(stem).group()
    # e_key = stem[:10]

    try :
        key = dict_ref[e_key]
    except :
        os.rename(join(path, f), join(path_noDB, f))
        noDB.append([path, f, path_noDB, f])
        continue

    try :
        new_name = key + " " + stem + add_file_name + ext   ####################
        success.append(re_name(join(path, f), join(path, new_name)))
    except Exception as e :
        print(e.__class__)
        fail.append([path, f, e.__class__, e])
        continue
loss = total - len(success) - len(fail) - len(noDB)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(fail)}개 파일 이름 변경 실패. {len(noDB)}개 전산없음')

write_log_csv(success, path_success, addInfo)
write_log_csv(noDB, path_noDB, addInfo)
write_log_csv(fail, path, addInfo)

100%|██████████| 49/49 [00:00<00:00, 2427.32it/s]

0개 파일 누수
49개 중 49개 파일 이름 변경 완료. 0개 파일 이름 변경 실패. 0개 전산없음
log 파일이 비어있습니다. path : D:\전산부재\광주 통합 6.KS KS양통
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\KS자산관리 양통\양통 종적


###### 계좌번호:폴더명(한울가람)

In [ ]:
##############################################
path = r'D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = "광주 모음 대출원장 KS 펜타원인"
add_file_name = "" # 공백 하나
##############################################
path_noDB = join(r'D:\전산부재', addInfo)
path_noExcel = join(path_noDB, "noExcel")
path_completed = join(r"D:\업로드완료", addInfo)
dict_ref = dict

# p_admin = re.compile(r'((?<=\D)|\b)\d{10}((?=\D)|\b)') # 10자리
noDB = []
noExcel = []
success = []
fail = []
dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)

if not os.path.exists(path_noExcel) :
    os.makedirs(path_noExcel)

if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

for d in tqdm(dir_list, total=total) :
    
    try :
        key = dict_ref[d]
        os.rename(join(path, d), join(path, key))
        success.append([path, d, path, key])
    except :
        os.rename(join(path, d), join(path_noExcel, d))
        noExcel.append([path, d, path_noExcel, d])
        continue

    
loss = total - len(success) - len(fail) - len(noDB)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(fail)}개 파일 이름 변경 실패. {len(noDB)}개 전산없음')

write_log_csv(success, path_success, addInfo)
write_log_csv(noExcel, path_noExcel, addInfo)

100%|██████████| 85/85 [00:00<00:00, 4430.03it/s]

84개 파일 누수
85개 중 1개 파일 이름 변경 완료. 0개 파일 이름 변경 실패. 0개 전산없음


###### 폴더명 2단계

In [15]:
##############################################
path = r'D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류'
addInfo = "광주 모음 대출원장 KS 펜타원인"
add_file_name = "" # 공백 하나
dict_ref1 = dict
dict_ref2 = dict_rgt
##############################################
path_noDB = join(r'D:\전산부재', addInfo)
path_noExcel = join(path_noDB, "noExcel")
path_completed = join(r"D:\업로드완료", addInfo)

# p_admin = re.compile(r'((?<=\D)|\b)\d{10}((?=\D)|\b)') # 10자리
noDB = []
noExcel = []
success = []
fail = []
dir_list = [d.name for d in os.scandir(path) if d.is_dir()]
total=len(dir_list)

if not os.path.exists(path_noDB) :
    os.makedirs(path_noDB)

if not os.path.exists(path_noExcel) :
    os.makedirs(path_noExcel)

if not os.path.exists(path_completed) :
    os.makedirs(path_completed)

for d in tqdm(dir_list, total=total) :
    
    try : 
        sn = dict_ref1[d]   # 1차 변환 nokey에러시 엑셀자료가 부족할 수 있으니 확인 필요
    except :
        try :
            os.rename(join(path, d), join(path_noExcel, d))
            noExcel.append(d)
        except : 
            fail.append([path, d, path_noExcel, d, e.__class__])
            print(d, "noExcel파일로 이동 실패")
            continue
        continue
        
    try :
        key = dict_ref2[sn] # 2차 변환 nokey에러시 전산부재, 단 sn의 포맷문제는 아닌지 로그 확인 필요(특히 사업자는 dict에 0이 막 붙어있어서)
    except :
        try :
            os.rename(join(path, d), join(path_noDB, sn))
            noDB.append([path, d, path_noDB, sn])
        except :
            fail.append([path, d, path_noDB, sn, e.__class__])
            print(d, "noDB파일로 이동 실패")
            continue
        continue

    try : # key로 폴더명 변환
        os.rename(join(path, d), join(path, key))
        success.append([path, d, path, key])
    except :
        fail.append([path, d, path, key, e.__class__])
        continue

    
loss = total - len(success) - len(fail) - len(noDB)
print(f'{loss}개 파일 누수')
print(f'{total}개 중 {len(success)}개 파일 이름 변경 완료. {len(fail)}개 파일 이름 변경 실패. {len(noDB)}개 전산없음. {len(noExcel)}개 엑셀에없음')

write_log_csv(success, path_success, addInfo)
write_log_csv(noExcel, path_noExcel, addInfo)
write_log_csv(noDB, path_noDB, addInfo)
write_log_csv(fail, path, addInfo)

100%|██████████| 444/444 [00:00<00:00, 4958.31it/s]

0개 파일 누수
444개 중 304개 파일 이름 변경 완료. 0개 파일 이름 변경 실패. 140개 전산없음
log 파일이 비어있습니다. path : D:\전산부재\광주 모음 대출원장 KS 펜타원인\noExcel
log 파일이 비어있습니다. path : D:\구 스캔파일\광주 공유폴더\모음\대출원장\케이에스자산관리\펜타원인서류


##### 수정일을 파일명에

In [4]:
path = r'D:\extraKey\광주 공유폴더\모음\신창\신창지사\신창 업데이트\양통\키움 양도통지서\1901'
file_list = [f.name for f in os.scandir(path) if f.is_file() and (re.search(p_extension, f.name))]
total=len(file_list)
cnt = 0
for f in tqdm(file_list, total=total) :
    stem = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]
    mtime = getMtime(join(path,f))
    new_name = stem + " " + mtime + ext
    try :
        os.rename(join(path, f), join(path, new_name))
        cnt += 1
    except Exception as e :
        print(f, e.__class__, e, sep=" : ")
        pass



100%|██████████| 137/137 [00:00<00:00, 3314.16it/s]


### 회사 코드
##### 펜타

In [58]:
path = r"D:\extraKey\광주 공유폴더\모음\대출원장\케이에스자산관리\양통"
file_list = file_listFnc(path)
files = pd.Series(file_list)
files[0]

'20409094 8101001694 M1_6305202796617_WRB_Wfni_001.pdf'

In [59]:
files1 = files.str.replace("M[0-9]", "", regex=True)
files1[0]

'20409094 8101001694 _6305202796617_WRB_Wfni_001.pdf'

In [60]:
files1 = files1.str.replace("N[0-9]", "보증인", regex=True)

In [61]:
files1 = files1.str.replace("KAC-[가나다라마바사아자차카타파하]", "KAC", regex=True)

In [71]:
df = pd.concat([files, files1], axis=1)
df.columns = ["origin_name", "new_name"]
df.head()

,origin_name,new_name
0,20409094 8101001694 M1_6305202796617_WRB_Wfni_...,20409094 8101001694 _6305202796617_WRB_Wfni_00...
1,20409094 8101001694 M3_6305202796617_DNP_PENCO...,20409094 8101001694 _6305202796617_DNP_PENCO_0...
2,20409094 8101001694 M3_6305202796617_DNP_PENCO...,20409094 8101001694 _6305202796617_DNP_PENCO_0...
3,20421357 8101001323 M1_6807141657411_WRB_Wfni_...,20421357 8101001323 _6807141657411_WRB_Wfni_00...
4,20421357 8101001323 M2_6807141657411_Wfni_DNP_...,20421357 8101001323 _6807141657411_Wfni_DNP_00...


In [73]:
df.origin_name[0], df.new_name[0]

('20409094 8101001694 M1_6305202796617_WRB_Wfni_001.pdf',
 '20409094 8101001694 _6305202796617_WRB_Wfni_001.pdf')

In [74]:
addInfo = "광주 모음 대출원장 케이에스 펜타"
success, fail = [], []
total = len(df.index)
for k, v in tqdm(df.iterrows(), total=total) :
    src = join(path, str(v.origin_name))
    dst = join(path, str(v.new_name))

    if src != dst :
        try : 
            os.rename(src, dst)
            success.append([src, dst])
        except Exception as e :
            fail.append([src, dst, e.__class__])


print(f"{total}개 중 {len(success)}개 파일이름 변경, {len(fail)}개 파일 이름 변경 실패")
write_log_csv(success, path_log, addInfo+"양통이름변환")
write_log_csv(fail, path, "실패")

### 폴더에서 채무자키 가져오기

In [4]:
keyPath = r"\\192.168.0.75\스캔파일\구 스캔파일\광주 공유폴더\통합파일\(6.케이에스자산관리대부_헬프)\2.원인서류(대출신청서)"
keyList = [f.name for f in os.scandir(keyPath) if f.is_dir()]
print(len(keyList))
print(keyList[:5])

1082
['20409079 장대식', '20409081 최문환', '20409082 신동혁', '20409083 유상형', '20409084 이동규']


In [8]:
path = r"D:\extraKey\광주 공유폴더\모음\대출원장\원장스캔(엑셀파일있나물어봐..없으면 수작업....)\제출"
file_list = file_listFnc(path)

In [16]:
success = []
fail = []
noKey = []
for f in file_list :
    key = ""
    debtor_name = f.split("#")[0]

    isOne = 0
    for k in keyList :
        if debtor_name in k :
            key = k[:8]
            isOne += 1

    if isOne == 1 :
        new_name = key + " " + f
        try : 
            os.rename(join(path, f), join(path, new_name))
            success.append([path, f, path, new_name])
        except Exception as e : 
            fail.append([path, f, path, new_name, e.__class__])
    else : 
        noKey.append([f, isOne])
        
write_log_csv(success, path, "광주 모음 대출원장 원장스캔 제출")
write_log_csv(fail, path, "fail")
write_log_csv(noKey, path, "noKey")

TypeError: stat: path should be string, bytes, os.PathLike or integer, not list

In [23]:
# check 
path = r"D:\extraKey\광주 공유폴더\모음\대출원장\원장스캔(엑셀파일있나물어봐..없으면 수작업....)\제출"
file_list = file_listFnc(path)
cnt = 0
for f in file_list :
    temp = f.split("#")[0]
    key = temp[:8]
    name = temp[8:].strip()
    if dict_refer[key][2] == name :
        pass
    else :
        print(key, name, dict_refer[key][2])
        cnt += 1
print(cnt)

20428561 서호준 서호준(서호예)
20428561 서호준 서호준(서호예)
20428647 김광수630410 김광수
20428647 김광수630410 김광수
20428828 김영철580630 김영철
20428875 류지성 유지성(류지성)
20428944 (주)케이디세코 (구 신명비앤에프) ㈜케이디세코 (구 신명비앤에프)
20428952 (주)나눔공판장 ㈜나눔공판장
20428953 (주)서통LBTECH ㈜서통LBTECH
20428954 (주)에스아이상사 ㈜에스아이상사
20428955 (주)윈컴멀티미디어 ㈜윈컴멀티미디어
20428955 (주)윈컴멀티미디어 ㈜윈컴멀티미디어
20428956 (주)정공트레이딩 ㈜정공트레이딩
20428957 (주)제성인터내셔날 ㈜제성인터내셔날
20428958 (주)종영 ㈜종영
20428958 (주)종영 ㈜종영
20428958 (주)종영 ㈜종영
20428958 (주)종영 ㈜종영
20428959 (주)케이텍 ㈜케이텍
20428959 (주)케이텍 ㈜케이텍
20428959 (주)케이텍 ㈜케이텍
20429015 김종수430213 김종수
20429015 김종수430213 김종수
20429045 박찬수450606 박찬수
20429055 삼흥철강(주) 삼흥철강㈜
20429059 세민공영(주) 세민공영㈜
20429059 세민공영(주) 세민공영㈜
20429059 세민공영(주) 세민공영㈜
20429069 아시아헤르텔중석(주) 아시아헤르텔중석㈜
20429189 태인임업(주) 태인임업㈜
20429228 (주)진성키디오 ㈜진성키디오
20429230 송암실업(주) 송암실업㈜
20429253 (주)챌린지디앤씨 ㈜챌린지디앤씨
20429343 이승우6109 이승우
20429350 이준석7004 이준석
20429377 최강을 최예준(변경전최강을)
20429409 (주)세웅 ㈜세웅
20429420 윤병철700217 윤병철
20429423 박정원670125 박정원
20429424 윤호준700228 윤호준
20429482 이성재 가남전자
20429482 이성재 가남전자
2

(137, 0)

### 파일명 변경파일 되돌리기

In [8]:
temp = pd.read_csv(r"D:\0.전산\1.진행중과업\파일서버개편\log\20221102 170545_63_광주 모음 신창 지사 업데이트 양도양수계약서 키움 1901.csv", header=None, names=["dst", "src"])
temp

,dst,src
0,9008000022-1.pdf,dict['9008000022'] 9008000022-1 .pdf
1,9008000022-2.pdf,dict['9008000022'] 9008000022-2 .pdf
2,9008000042-1.pdf,dict['9008000042'] 9008000042-1 .pdf
3,9008000042-2.pdf,dict['9008000042'] 9008000042-2 .pdf
4,9008000042-3.pdf,dict['9008000042'] 9008000042-3 .pdf
...,...,...
58,9008000419-1.pdf,dict['9008000419'] 9008000419-1 .pdf
59,9008000419-2.pdf,dict['9008000419'] 9008000419-2 .pdf
60,9008000419-3.pdf,dict['9008000419'] 9008000419-3 .pdf
61,9008000421-1.pdf,dict['9008000421'] 9008000421-1 .pdf


In [9]:
for i, v in temp.iterrows() :
    os.rename(join(path, v.src), join(path, v.dst))

In [23]:
f = "2001가소12345"
p_event1=re.compile(r"((?<=\D)|^)(19\d\d|20[012]\d)\s?(준?재?[가간나느다머차카타즈본징하개회라][가-힣]{0,2})\s?([0-9]+)")
p_event2=re.compile(r"((?<=\D)|^)([01289]\d)\s?(준?재?[가간나느다머차카타즈본징하개회라][가-힣]{0,2})\s?([0-9]+)")
p_event3=re.compile(r"((?<=\D)|^)(19\d\d|20[012]\d)(준?재?[가간나느다머차카타즈본징하개회라][가-힣]{0,2})\b") #반드시 띄어쓰기를 하거나 끝이나야.(& 1보다 먼저 if문에서 나오면 안됨.)
p_event4=re.compile(r"((?<=\D)|^)([01289]\d)(준?재?[가간나느다머차카타즈본징하개회라][가-힣]{0,2})\b") #반드시 띄어쓰기를 하거나 끝이나야.(& 1보다 먼저 if문에서 나오면 안됨.)
p_event1.search(f)

<re.Match object; span=(0, 11), match='2001가소12345'>

In [8]:
from datetime import datetime
year = datetime.today().year

In [18]:
years = [str(i) for i in (range(2000, year+1))]

In [19]:
years_pattern = "|".join(years)
years_pattern

'2000|2001|2002|2003|2004|2005|2006|2007|2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022'

In [39]:
p = re.compile("((?<=\D)|^)(19\d\d|"+years_pattern+")\s?(준?재?[가간나느다머차카타즈본징하개회라][가-힣]{0,2})\s?([0-9]+)")
p.search("1956가소12345")

<re.Match object; span=(0, 11), match='1956가소12345'>